In [17]:
import re
import time
import pymysql
import pandas as pd
from translate_api.translate_api import api
from numpy import *
import unidecode
import snowballstemmer
from hunspell import Hunspell
import numpy as np

def levenshtein(source, target):
    if len(source) < len(target):
        return levenshtein(target, source)

    # So now we have len(source) >= len(target).
    if len(target) == 0:
        return len(source)

    # We call tuple() to force strings to be used as sequences
    # ('c', 'a', 't', 's') - numpy uses them as values by default.
    source = np.array(tuple(source))
    target = np.array(tuple(target))

    # We use a dynamic programming algorithm, but with the
    # added optimization that we only need the last two rows
    # of the matrix.
    previous_row = np.arange(target.size + 1)
    for s in source:
        # Insertion (target grows longer than source):
        current_row = previous_row + 1

        # Substitution or matching:
        # Target and source items are aligned, and either
        # are different (cost of 1), or are the same (cost of 0).
        current_row[1:] = np.minimum(
                current_row[1:],
                np.add(previous_row[:-1], target != s))

        # Deletion (target grows shorter than source):
        current_row[1:] = np.minimum(
                current_row[1:],
                current_row[0:-1] + 1)

        previous_row = current_row

    return previous_row[-1]



h = Hunspell('pt_BR', hunspell_data_dir='pt_BR')

read = pd.ExcelFile('Banco de Termos final Lucas 10-04-2019.xlsx')

In [18]:
df_termos = read.parse('Planilha1')

In [19]:
read = pd.ExcelFile('CliniMap_CIPE2.xlsx')
CIPE_df = read.parse('Match')

In [20]:
read = pd.ExcelFile('CIPE2017_tabela organizada_CPeCC.xlsx')
df_CIPE = read.parse('Plan1')

In [5]:
def comb(lis, dict_lis):
    aux = []
    for s in lis:
        for ss in dict_lis:
            aux.append(s+' '+ss)
    return aux

In [6]:
def removearticles(text):
    text = re.sub('"','',text)
    return re.sub('\s+(de|do|dos|da|das|o|os|a|as|com|para|pra|e|ou|na|nas|no|nos|ao|aos|em|mg|ml|um)(\s+)',' ', text)

In [7]:
def lematizador(text):
    auxt = str('')
    words = text.split()
    for x in words:
        if h.spell(x):
            if len(h.stem(x)) != 0:
                auxt += h.stem(x)[0] + ' '
            else:
                auxt += x + ' '
        else:
            auxt += x + ' '
    auxt.strip()
    return auxt

In [8]:
def stemmer(text):
    stemmer = snowballstemmer.stemmer('portuguese')
    return ' '.join(stemmer.stemWords(text.split())).strip()

In [9]:
ini = time.time()
h = Hunspell('pt_BR', hunspell_data_dir='pt_BR')


cnx = pymysql.connect(user='root', password='1234',database='umls2017aa')
cursor = cnx.cursor()
sqlstr = ("SELECT  CUI, STR FROM umls2017aa.mrconso where LAT='POR' order by CUI;")
cursor.execute(sqlstr)
umls = []
CUI1 = []
STR1 = []
y = 0
for (CUI,STR) in cursor:
    CUI1.insert(y,CUI)
    STR1.insert(y,STR)
    y+=1
umls = array([STR1,CUI1])
umls2 = array([STR1,STR1,STR1,CUI1])
df_umls = pd.DataFrame(umls.transpose(),columns=['STR','CUI'])
df_umls_norm = pd.DataFrame(umls2.transpose(),columns=['STR', 'STRlemma', 'STRstem', 'CUI'])

sqlstr = ("SELECT  CUI, WD FROM umls2017aa.mrxw_por;")
cursor.execute(sqlstr)
umls = []
CUI2 = []
WD2 = []
y = 0
for (CUI,WD) in cursor:
    CUI2.insert(y,CUI)
    WD2.insert(y,WD)
    y+=1
umls_word = array([WD2,CUI2])
umls2_word = array([WD2,WD2,WD2,CUI2])
df_umls_word = pd.DataFrame(umls_word.transpose(),columns=['WD','CUI'])
df_umls_word_norm = pd.DataFrame(umls2_word.transpose(),columns=['WD', 'WDlemma', 'WDstem', 'CUI'])

sqlstr = ("SELECT CUI1, CUI2 FROM umls2017aa.mrrel where CUI1!=CUI2 and REL='SY';")
cursor.execute(sqlstr)
umls = []
CUIs2 = []
CUIs1 = []
y = 0
for (CUI1,CUI2) in cursor:
    CUIs1.insert(y,CUI1)
    CUIs2.insert(y,CUI2)
    y+=1
umls_word = array([CUIs2,CUIs1])
df_umls_rel = pd.DataFrame(umls_word.transpose(),columns=['CUI1','CUI2'])

cursor.close()
cnx.close()

for x in range(0,df_umls['STR'].size):
    df_umls.at[x,'STR'] = str(df_umls.at[x,'STR']).lower()

for x in range(0,df_umls_word['WD'].size):
    df_umls_word.at[x,'WD'] = str(df_umls_word.at[x,'WD']).lower()

for x in range(0,df_umls_norm['STR'].size):
    df_umls_norm.at[x,'STR'] = str(df_umls_norm.at[x,'STR']).lower()
    df_umls_norm.at[x,'STRlemma'] = str(df_umls_norm.at[x,'STRlemma']).lower()
    df_umls_norm.at[x,'STRstem'] = str(df_umls_norm.at[x,'STRstem']).lower()

for x in range(0,df_umls_word_norm['WD'].size):
    df_umls_word_norm.at[x,'WD'] = str(df_umls_word_norm.at[x,'WD']).lower()
    df_umls_word_norm.at[x,'WDlemma'] = str(df_umls_word_norm.at[x,'WDlemma']).lower()
    df_umls_word_norm.at[x,'WDstem'] = str(df_umls_word_norm.at[x,'WDstem']).lower()


for x in range(0,df_umls_norm['STR'].size):
    df_umls_norm.at[x,'STR'] = removearticles(unidecode.unidecode(str(df_umls_norm.at[x,'STR']))).strip()

for x in range(0,df_umls_norm['STRlemma'].size):
    df_umls_norm.at[x,'STRlemma'] = removearticles(unidecode.unidecode(lematizador(str(df_umls_norm.at[x,'STRlemma']))))

for x in range(0,df_umls_norm['STRstem'].size):
    df_umls_norm.at[x,'STRstem'] = removearticles(unidecode.unidecode(stemmer(str(df_umls_norm.at[x,'STRstem']))))

for x in range(0,df_umls_word_norm['WD'].size):
    df_umls_word_norm.at[x,'WD'] = unidecode.unidecode(str(df_umls_word_norm.at[x,'WD'])).strip()

for x in range(0,df_umls_word_norm['WDlemma'].size):
    df_umls_word_norm.at[x,'WDlemma'] = unidecode.unidecode(lematizador(str(df_umls_word_norm.at[x,'WDlemma'])))

for x in range(0,df_umls_word_norm['WDstem'].size):
    df_umls_word_norm.at[x,'WDstem'] = unidecode.unidecode(stemmer(str(df_umls_word_norm.at[x,'WDstem'])))

In [21]:
for x in range(0,df_termos.size):
    df_termos.at[x,'Termos'] = str(df_termos.at[x,'Termos']).lower()

In [22]:
for x in range(0,df_CIPE['Termo'].size):
    df_CIPE.at[x,'Termo'] = str(df_CIPE.at[x,'Termo']).lower()

In [23]:
df_termos_norm = pd.DataFrame(data=None, columns=['Termos'])
df_termos_lemma = pd.DataFrame(data=None, columns=['Termos'])
df_termos_stem = pd.DataFrame(data=None, columns=['Termos'])

In [24]:
df_CIPE_norm = pd.DataFrame(data=None, columns=['Termo'])
df_CIPE_lemma = pd.DataFrame(data=None, columns=['Termo'])
df_CIPE_stem = pd.DataFrame(data=None, columns=['Termo'])

In [25]:
for x in range(0,df_termos.size):
    df_termos_norm.at[x,'Termos'] = removearticles(unidecode.unidecode(str(df_termos.at[x,'Termos']))).strip()

In [26]:
for x in range(0,df_termos.size):
    df_termos_lemma.at[x,'Termos'] = removearticles(unidecode.unidecode(lematizador(str(df_termos.at[x,'Termos']))))

In [27]:
for x in range(0,df_termos.size):
    df_termos_stem.at[x,'Termos'] = removearticles(unidecode.unidecode(stemmer(str(df_termos.at[x,'Termos']))))

In [28]:
for x in range(0,df_CIPE['Termo'].size):
    df_CIPE_norm.at[x,'Termo'] = removearticles(unidecode.unidecode(str(df_CIPE.at[x,'Termo']))).strip()

In [29]:
for x in range(0,df_CIPE['Termo'].size):
    df_CIPE_lemma.at[x,'Termo'] = removearticles(unidecode.unidecode(lematizador(str(df_CIPE.at[x,'Termo']))))

In [30]:
for x in range(0,df_CIPE['Termo'].size):
    df_CIPE_stem.at[x,'Termo'] = removearticles(unidecode.unidecode(stemmer(str(df_CIPE.at[x,'Termo']))))

In [31]:
print(df_termos.at[301,'Termos'])
print(df_termos_norm.at[301,'Termos'])
print(df_termos_lemma.at[301, 'Termos'])
print(df_termos_stem.at[301, 'Termos'])

estagnação
estagnacao
estagnacao 
estagn


In [32]:
print(df_CIPE.at[2301,'Termo'])
print(df_CIPE_norm.at[2301,'Termo'])
print(df_CIPE_lemma.at[2301, 'Termo'])
print(df_CIPE_stem.at[2301, 'Termo'])

controle da dor 
controle dor
controlar dor 
control dor


In [33]:
from dicio import Dicio

# Create a Dicio object
dicio = Dicio()

# Search for "Doce" and return an object Word
word = dicio.search("leito")

# Print the word, the url and the meaning
# print(word, word.url, word.meaning)

# Print a list of synonyms
print(word.synonyms)

[cama, álveo, casamento, matrimônio, estrato]


In [34]:
df_termos_match = pd.DataFrame(data=None, columns=['Termos', 'Regra', 'perc', 'ICNPcode', 'ICNPterm',
                                                   'ICNPmod', 'ICNPeixo', 'ICNPVersao'])

In [34]:
print(100 - 1 / len('lucass') * 100)

zz = 0
achou = 0
candidato = ''
perc_max = 0
metodo = 0
y_aux = 0
terms = 0



controle = 1
for x in range(0, len(df_termos_norm)):
    CUIS = []
    t = len(df_termos_norm.at[x,'Termos'])
    t_max = int(t+(t*0.2))
    t_min =  int(t-(t*0.2))
    t_lemma = len(df_termos_lemma.at[x, 'Termos'])
    t_max_lemma = int(t_lemma+(t_lemma*0.2))
    t_min_lemma =  int(t_lemma-(t_lemma*0.2))
    perc_max=0
    metodo = 0
    '''locTerm = CIPE_df.loc[CIPE_df['Termos'] == df_termos.at[x, 'Termos']]
    if not locTerm['Termos'].empty:
    #if df_termos.at[x, 'Termos'] == CIPE_df.at[zzz, 'Termos']:
        for xx in range(0,len(locTerm['Termos'])):
            df_termos_match.at[zz, 'Termos'] = CIPE_df.at[locTerm.index[xx], 'Termos']
            df_termos_match.at[zz, 'ICNPcode'] = CIPE_df.at[locTerm.index[xx], 'ICNPcode']
            df_termos_match.at[zz, 'ICNPterm'] = CIPE_df.at[locTerm.index[xx], 'ICNPterm']
            df_termos_match.at[zz, 'ICNPmod'] = CIPE_df.at[locTerm.index[xx], 'ICNPmod']
            df_termos_match.at[zz, 'ICNPVersao'] = CIPE_df.at[locTerm.index[xx], 'ICNPVersao']
            df_termos_match.at[zz, 'ICNPeixo'] = CIPE_df.at[locTerm.index[xx], 'ICNPeixo']
            df_termos_match.at[zz, 'Regra'] = CIPE_df.at[locTerm.index[xx], 'Regra']
            df_termos_match.at[zz, 'perc'] = CIPE_df.at[locTerm.index[xx], 'perc']
            zz += 1
        continue
    print(df_termos.at[x, 'Termos'], ' ------- passou')
    print(controle , ' termos ja passaram')'''
    controle += 1
    for y in range(0, len(df_CIPE_norm)):
        num = levenshtein(df_termos_norm.at[x, 'Termos'], df_CIPE_norm.at[y, 'Termo'])
        perc = 100 - num / len(df_CIPE_norm.at[y, 'Termo']) * 100
        if perc > perc_max:
            perc_max = perc
            candidato = df_CIPE_norm.at[y, 'Termo']
            metodo = 1
            y_aux = y
    if not perc_max == 100:
        print(df_termos_norm.at[x,'Termos'],' entrou')
        noventa = False
        for y in range(0,len(df_umls_norm['STR'])):
            
            if df_termos_norm.at[x,'Termos'][0] == df_umls_norm.at[y,'STR'][0] and len(df_umls_norm.at[y,'STR']) < t_max and len(df_umls_norm.at[y,'STR']) > t_min:
                num = levenshtein(df_termos_norm.at[x,'Termos'],df_umls_norm.at[y,'STR'])
                if len(df_umls_norm.at[y,'STR']) > 0:
                    perc = 100-num/len(df_umls_norm.at[y,'STR'])*100
                else:
                    perc = 0
                if perc == 100:
                    if not df_umls_norm.at[y, 'CUI'] in CUIS:
                        CUIS.append(df_umls_norm.at[y, 'CUI'])
                        for z in range(0, len(df_umls_rel)):
                            if df_umls_rel.at[z,'CUI1'] == df_umls_norm.at[y, 'CUI']:
                                if not df_umls_rel.at[z,'CUI2'] in CUIS:
                                    CUIS.append(df_umls_rel.at[z,'CUI2'])
                    print('achou')
                    noventa = False
                    terms += 1
                    break
                if perc >= 90:
                    if not df_umls_norm.at[y, 'CUI'] in CUIS:
                        CUIS.append(df_umls_norm.at[y, 'CUI'])
                        for z in range(0, len(df_umls_rel)):
                            if df_umls_rel.at[z,'CUI1'] == df_umls_norm.at[y, 'CUI']:
                                if not df_umls_rel.at[z,'CUI2'] in CUIS:
                                    CUIS.append(df_umls_rel.at[z,'CUI2'])
                    print('talvez achou')
                    noventa = True
        if noventa:
            terms+=1
        if perc < 90:
            noventa = False
        for y in range(0,len(df_umls_norm['STR'])):
            if df_termos_lemma.at[x, 'Termos'][0] == df_umls_norm.at[y, 'STRlemma'][0] and len(df_umls_norm.at[y, 'STRlemma']) < t_max_lemma and len(df_umls_norm.at[y, 'STRlemma']) > t_min_lemma:
                num_lemma = levenshtein(df_termos_lemma.at[x, 'Termos'], df_umls_norm.at[y, 'STRlemma'])
                num_steam = levenshtein(df_termos_stem.at[x, 'Termos'], df_umls_norm.at[y, 'STRstem'])
                if len(df_umls_norm.at[y, 'STRlemma']) > 0:
                    perc_lemma = 100-num_lemma/len(df_umls_norm.at[y, 'STRlemma'])*100
                    perc_steam = 100-num_steam/len(df_umls_norm.at[y, 'STRstem'])*100
                else:
                    perc_lemma = 0
                    perc_steam = 0
                if perc_lemma == 100 and perc_steam >= 90:
                    if not df_umls_norm.at[y, 'CUI'] in CUIS:
                        CUIS.append(df_umls_norm.at[y, 'CUI'])
                        for z in range(0, len(df_umls_rel)):
                            if df_umls_rel.at[z,'CUI1'] == df_umls_norm.at[y, 'CUI']:
                                if not df_umls_rel.at[z,'CUI2'] in CUIS:
                                    CUIS.append(df_umls_rel.at[z,'CUI2'])
                    print('achou')
                    noventa = False
                    terms += 1
                    break
                if perc_lemma >= 90 and perc_steam >= 90:
                    if not df_umls_norm.at[y, 'CUI'] in CUIS:
                        CUIS.append(df_umls_norm.at[y, 'CUI'])
                        for z in range(0, len(df_umls_rel)):
                            if df_umls_rel.at[z,'CUI1'] == df_umls_norm.at[y, 'CUI']:
                                if not df_umls_rel.at[z,'CUI2'] in CUIS:
                                    CUIS.append(df_umls_rel.at[z,'CUI2'])
                    print('talvez achou')
                    noventa = True
        if noventa:
            terms+=1
        if len(CUIS) > 0:
            print('existe na UMLS com as CUIs ', CUIS)
            for c in CUIS:
                perc = 0
                perc_max = 0
                print(c)
                locCUI = df_umls_norm.loc[df_umls_norm['CUI'] == c]
                print(locCUI)
                if not locCUI['CUI'].empty:
                    for xx in range(0,len(locCUI['CUI'])):
                        perc = 0
                        perc_max = 0
                        df_umls_norm.at[locCUI.index[xx], 'STR']
                        df_umls_norm.at[locCUI.index[xx], 'STRlemma']
                        df_umls_norm.at[locCUI.index[xx], 'STRstem']
                        
                        for y in range(0, len(df_CIPE_norm)):
                            num = levenshtein(df_umls_norm.at[locCUI.index[xx], 'STR'], df_CIPE_norm.at[y, 'Termo'])
                            perc = 100 - num / len(df_CIPE_norm.at[y, 'Termo']) * 100
                            if perc > perc_max:
                                perc_max = perc
                                candidato = df_CIPE_norm.at[y, 'Termo']
                                metodo = 1
                                y_aux = y
                            if perc_max == 100:
                                metodo = 1
                                break
                        if metodo == 1 and perc_max >= 90:
                            print('encontrou o termo ', df_umls_norm.at[locCUI.index[xx], 'STR'], ' na CIPE o termo ', df_CIPE.at[y_aux, 'Termo'], ' com o codigo CIPE ',df_CIPE.at[y_aux, 'Código'])
                            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                            df_termos_match.at[zz, 'ICNPmod'] = df_umls_norm.at[locCUI.index[xx], 'STR']
                            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                            df_termos_match.at[zz, 'Regra'] = 1
                            df_termos_match.at[zz, 'perc'] = perc_max
                            zz += 1
                        if not perc_max == 100:
                            for y in range(0, len(df_CIPE_norm)):
                                numlemma = levenshtein(df_umls_norm.at[locCUI.index[xx], 'STRlemma'], df_CIPE_lemma.at[y, 'Termo'])
                                numstem = levenshtein(df_umls_norm.at[locCUI.index[xx], 'STRstem'], df_CIPE_stem.at[y, 'Termo'])
                                perclemma = 100 - numlemma / len(df_CIPE_lemma.at[y, 'Termo']) * 100
                                percstem = 100 - numstem / len(df_CIPE_lemma.at[y, 'Termo']) * 100
                                if perclemma == 100 and percstem == 100:
                                    print('encontrou o termo ', df_umls_norm.at[locCUI.index[xx], 'STR'], ' na CIPE o termo ', df_CIPE.at[y_aux, 'Termo'], ' com o codigo CIPE ',df_CIPE.at[y_aux, 'Código'])
                                    df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                    df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y, 'Código']
                                    df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y, 'Termo']
                                    df_termos_match.at[zz, 'ICNPmod'] = df_umls_norm.at[locCUI.index[xx], 'STR']
                                    df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y, 'Versão']
                                    df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y, 'Eixo']
                                    df_termos_match.at[zz, 'Regra'] = 2
                                    df_termos_match.at[zz, 'perc'] = 100
                                    zz += 1
                                elif perclemma >= 90 and percstem >= 90:
                                    print('encontrou o termo ', df_umls_norm.at[locCUI.index[xx], 'STR'], ' na CIPE o termo ', df_CIPE.at[y_aux, 'Termo'], ' com o codigo CIPE ',df_CIPE.at[y_aux, 'Código'])
                                    df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                    df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y, 'Código']
                                    df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y, 'Termo']
                                    df_termos_match.at[zz, 'ICNPmod'] = df_umls_norm.at[locCUI.index[xx], 'STR']
                                    df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y, 'Versão']
                                    df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y, 'Eixo']
                                    df_termos_match.at[zz, 'Regra'] = 2
                                    df_termos_match.at[zz, 'perc'] = (perclemma,percstem)
                                    zz += 1
print('foram encontrados pelo metodo 1, ', terms,' termos')

83.33333333333334
abertura indiscriminada  entrou


KeyboardInterrupt: 

In [51]:
writer = pd.ExcelWriter('CliniMap_CIPE_sin_UMLS2.xlsx')
df_termos_match.to_excel(writer, 'Match')
writer.save()

In [35]:
print(100 - 1 / len('lucass') * 100)

zz = 0
achou = 0
candidato = ''
perc_max = 0
metodo = 0
y_aux = 0


controle = 1
for x in range(0, len(df_termos_norm)):
    
    metodo = 0
    '''locTerm = CIPE_df.loc[CIPE_df['Termos'] == df_termos.at[x, 'Termos']]
    if not locTerm['Termos'].empty:
    #if df_termos.at[x, 'Termos'] == CIPE_df.at[zzz, 'Termos']:
        for xx in range(0,len(locTerm['Termos'])):
            df_termos_match.at[zz, 'Termos'] = CIPE_df.at[locTerm.index[xx], 'Termos']
            df_termos_match.at[zz, 'ICNPcode'] = CIPE_df.at[locTerm.index[xx], 'ICNPcode']
            df_termos_match.at[zz, 'ICNPterm'] = CIPE_df.at[locTerm.index[xx], 'ICNPterm']
            df_termos_match.at[zz, 'ICNPmod'] = CIPE_df.at[locTerm.index[xx], 'ICNPmod']
            df_termos_match.at[zz, 'ICNPVersao'] = CIPE_df.at[locTerm.index[xx], 'ICNPVersao']
            df_termos_match.at[zz, 'ICNPeixo'] = CIPE_df.at[locTerm.index[xx], 'ICNPeixo']
            df_termos_match.at[zz, 'Regra'] = CIPE_df.at[locTerm.index[xx], 'Regra']
            df_termos_match.at[zz, 'perc'] = CIPE_df.at[locTerm.index[xx], 'perc']
            zz += 1
        continue
    print(df_termos.at[x, 'Termos'], ' ------- passou')
    print(controle , ' termos ja passaram')'''
    controle += 1
    for y in range(0, len(df_CIPE_norm)):
        num = levenshtein(df_termos_norm.at[x, 'Termos'], df_CIPE_norm.at[y, 'Termo'])
        perc = 100 - num / len(df_CIPE_norm.at[y, 'Termo']) * 100
        if perc > perc_max:
            perc_max = perc
            candidato = df_CIPE_norm.at[y, 'Termo']
            metodo = 1
            y_aux = y
        if perc_max == 100:
            metodo = 1
            break
    if metodo == 1 and perc_max >= 90:
        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
        df_termos_match.at[zz, 'ICNPmod'] = candidato
        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
        df_termos_match.at[zz, 'Regra'] = 1
        df_termos_match.at[zz, 'perc'] = perc_max
        zz += 1
    if not perc_max == 100:
        for y in range(0, len(df_CIPE_norm)):
            num = levenshtein(df_termos_lemma.at[x, 'Termos'], df_CIPE_lemma.at[y, 'Termo'])
            perc = 100 - num / len(df_CIPE_lemma.at[y, 'Termo']) * 100
            if perc > perc_max:
                perc_max = perc
                candidato = df_CIPE_lemma.at[y, 'Termo']
                metodo = 2
                y_aux = y
            elif perc == 100 and perc_max == 100:
                df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                df_termos_match.at[zz, 'ICNPmod'] = candidato
                df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                df_termos_match.at[zz, 'Regra'] = 2
                df_termos_match.at[zz, 'perc'] = perc_max
                zz += 1
                perc_max = perc
                candidato = df_CIPE_lemma.at[y, 'Termo']
                metodo = 2
                y_aux = y
        if metodo == 2 and perc_max >= 90:
            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
            df_termos_match.at[zz, 'ICNPmod'] = candidato
            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
            df_termos_match.at[zz, 'Regra'] = 2
            df_termos_match.at[zz, 'perc'] = perc_max
            zz += 1
        #if not perc_max == 100:
        if not perc_max == 1000:
            perc_max = 0
            for y in range(0, len(df_CIPE_norm)):
                num = levenshtein(df_termos_stem.at[x, 'Termos'], df_CIPE_stem.at[y, 'Termo'])
                perc = 100 - num / len(df_CIPE_stem.at[y, 'Termo']) * 100
                if perc > perc_max:
                    perc_max = perc
                    candidato = df_CIPE_stem.at[y, 'Termo']
                    metodo = 3
                    y_aux = y
                elif perc == 100 and perc_max == 100:
                    df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                    df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                    df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                    df_termos_match.at[zz, 'ICNPmod'] = candidato
                    df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                    df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                    df_termos_match.at[zz, 'Regra'] = 3
                    df_termos_match.at[zz, 'perc'] = perc_max
                    zz += 1
                    perc_max = perc
                    candidato = df_CIPE_lemma.at[y, 'Termo']
                    metodo = 3
                    y_aux = y
            if metodo == 3 and perc_max >= 90:
                df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                df_termos_match.at[zz, 'ICNPmod'] = candidato
                df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                df_termos_match.at[zz, 'Regra'] = 3
                df_termos_match.at[zz, 'perc'] = perc_max
                zz += 1
            #if perc_max < 90:
            if perc_max < 1000:
                perc_max = 0
                sem_sin = False
                sinonimos_t = []
                if len(df_termos.at[x, 'Termos'].split()) == 1:
                    try:
                        word = dicio.search(df_termos.at[x, 'Termos'])
                        for w in word.synonyms:
                            sinonimos_t.append(removearticles(unidecode.unidecode(str(w))).strip())
                    except:
                        sem_sin = True
                else:
                    iniR = time.time()
                    sin = {}
                    sin_lemma = {}
                    sin_steam = {}
                    words2 = (removearticles(df_termos.at[x, 'Termos']).strip()).split()
                    print(words2)
                    for p in words2:
                        sinonimos = []
                        sinonimos_lemma = []
                        sinonimos_steam = []
                        try:
                            w = dicio.search(p)
                            sinonimos = w.synonyms
                            i = 0
                            for s in sinonimos:
                                sinonimos[i] = unidecode.unidecode(str(s))
                                i += 1
                            i = 0
                            for s in sinonimos:
                                sinonimos_lemma.append(lematizador(str(s)))
                                sinonimos_steam.append(stemmer(str(s)))
                                i += 1
                            sinonimos.append(p)
                            sinonimos_lemma.append(lematizador(p))
                            sinonimos_steam.append(stemmer(p))
                            sin[p] = sinonimos
                            sin_lemma[p] = sinonimos_lemma
                            sin_steam[p] = sinonimos_steam
                        except:
                            sin[p]=[p]
                            sin_lemma[p] = [lematizador(p)]
                            sin_steam[p] = [stemmer(p)]
                    fimR = time.time()
                    print('tempo sinonimos, ', (fimR-iniR))
                    print('---------dicionarios-------', df_termos_norm.at[x,'Termos'])
                    print(sin)
                    print(sin_lemma)
                    print(sin_steam)


                    n = 0
                    first = True
                    for p in words2:
                        if first:
                            n = len(sin[p])
                            first = False
                            continue
                        else:
                            if len(sin[p]) != 0:
                                n *= len(sin[p])
                            else:
                                continue
                    print(df_termos_norm.at[x, 'Termos'], ' numero de combinações: ', n)
                    posib = []
                    posib_lemma = []
                    posib_steam = []
                    
                    
                    iniR = time.time()
                    first = True
                    for p in words2:
                        if first:
                            first = False
                            for s in sin[p]:
                                posib.append(s)
                            for s in sin_lemma[p]:
                                posib_lemma.append(s)
                            for s in sin_steam[p]:
                                posib_steam.append(s)
                        else:
                            posib = comb(posib, sin[p])
                            posib_lemma = comb(posib_lemma, sin_lemma[p])
                            posib_steam = comb(posib_steam, sin_steam[p])
                    fimR = time.time()
                    print('tempo posib, ', (fimR-iniR))
                    print(posib)
                    
                    if len(posib) > 1:
                        sinonimos_t = posib
                    else:
                        sem_sin = True
                    
                if not sem_sin:
                    for sin in sinonimos_t:
                        for y in range(0, len(df_CIPE_norm)):
                            num = levenshtein(str(sin), df_CIPE_norm.at[y, 'Termo'])
                            perc = 100 - num / len(df_CIPE_norm.at[y, 'Termo']) * 100
                            if perc > perc_max:
                                perc_max = perc
                                candidato = df_CIPE_norm.at[y, 'Termo']
                                metodo = 12
                                y_aux = y
                            if perc_max == 100:
                                metodo = 12
                                break
                        if not perc_max == 100:
                            for y in range(0, len(df_CIPE_norm)):
                                num = levenshtein(lematizador(str(sin)), df_CIPE_lemma.at[y, 'Termo'])
                                perc = 100 - num / len(df_CIPE_lemma.at[y, 'Termo']) * 100
                                if perc > perc_max:
                                    perc_max = perc
                                    candidato = df_CIPE_lemma.at[y, 'Termo']
                                    metodo = 22
                                    y_aux = y
                                elif perc == 100 and perc_max == 100:
                                    df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                    df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                    df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                    df_termos_match.at[zz, 'ICNPmod'] = candidato
                                    df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                    df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                    df_termos_match.at[zz, 'Regra'] = 22
                                    df_termos_match.at[zz, 'perc'] = perc_max
                                    zz += 1
                                    perc_max = perc
                                    candidato = df_CIPE_lemma.at[y, 'Termo']
                                    metodo = 22
                                    y_aux = y
                            if not perc_max == 100:
                                for y in range(0, len(df_CIPE_norm)):
                                    num = levenshtein(stemmer(str(sin)), df_CIPE_stem.at[y, 'Termo'])
                                    perc = 100 - num / len(df_CIPE_stem.at[y, 'Termo']) * 100
                                    if perc > perc_max:
                                        perc_max = perc
                                        candidato = df_CIPE_stem.at[y, 'Termo']
                                        metodo = 32
                                        y_aux = y
                                    elif perc == 100 and perc_max == 100:
                                        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                        df_termos_match.at[zz, 'ICNPmod'] = candidato
                                        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                        df_termos_match.at[zz, 'Regra'] = 32
                                        df_termos_match.at[zz, 'perc'] = perc_max
                                        zz += 1
                                        perc_max = perc
                                        candidato = df_CIPE_lemma.at[y, 'Termo']
                                        metodo = 32
                                        y_aux = y
                    if metodo == 12 and perc_max >= 90:
                        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                        df_termos_match.at[zz, 'ICNPmod'] = candidato
                        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                        df_termos_match.at[zz, 'Regra'] = 12
                        df_termos_match.at[zz, 'perc'] = perc_max
                        zz += 1
                    elif metodo == 22 and perc_max >= 90:
                        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                        df_termos_match.at[zz, 'ICNPmod'] = candidato
                        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                        df_termos_match.at[zz, 'Regra'] = 22
                        df_termos_match.at[zz, 'perc'] = perc_max
                        zz += 1
                    elif metodo == 32 and perc_max >= 90:
                        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                        df_termos_match.at[zz, 'ICNPmod'] = candidato
                        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                        df_termos_match.at[zz, 'Regra'] = 32
                        df_termos_match.at[zz, 'perc'] = perc_max
                        zz += 1
                #if not (metodo == 12 or metodo == 22 or metodo == 32):
                if not (metodo == 122 or metodo == 222 or metodo == 322):
                    perc_max = 0
                    words = df_termos_norm.at[x, 'Termos'].split()
                    if len(words) > 1:
                        for y in range(0, len(df_CIPE_norm)):
                            word_comp = df_CIPE_norm.at[y, 'Termo'].split()
                            med_perc = 0
                            if len(words) < len(word_comp):
                                subString = True
                                for p in words:
                                    tem_word = False
                                    for p_comp in word_comp:
                                        num = levenshtein(p, p_comp)
                                        perc = 100 - num / len(p_comp) * 100
                                        if perc > 90:
                                            med_perc += perc
                                            tem_word = True
                                            break
                                    if not tem_word:
                                        subString = False
                                if subString:
                                    perc_max = med_perc / len(words)
                                    candidato = df_CIPE_norm.at[y, 'Termo']
                                    metodo = 51
                                    y_aux = y
                                    df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                    df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                    df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                    df_termos_match.at[zz, 'ICNPmod'] = candidato
                                    df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                    df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                    df_termos_match.at[zz, 'Regra'] = 51
                                    df_termos_match.at[zz, 'perc'] = perc_max
                                    zz += 1
                            elif len(words) > len(word_comp):
                                supString = True
                                for p in word_comp:
                                    tem_word = False
                                    for p_comp in words:
                                        num = levenshtein(p, p_comp)
                                        perc = 100 - num / len(p_comp) * 100
                                        if perc > 90:
                                            med_perc += perc
                                            tem_word = True
                                            break
                                    if not tem_word:
                                        supString = False
                                if supString:
                                    perc_max = med_perc / len(word_comp)
                                    candidato = df_CIPE_norm.at[y, 'Termo']
                                    metodo = 61
                                    y_aux = y
                                    df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                    df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                    df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                    df_termos_match.at[zz, 'ICNPmod'] = candidato
                                    df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                    df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                    df_termos_match.at[zz, 'Regra'] = 61
                                    df_termos_match.at[zz, 'perc'] = perc_max
                                    zz += 1
                    elif len(words) == 1:
                        for y in range(0, len(df_CIPE_norm)):
                            word_comp = df_CIPE_norm.at[y, 'Termo'].split()
                            med_perc = 0
                            if len(words) < len(word_comp):
                                subString = True
                                for p in words:
                                    tem_word = False
                                    for p_comp in word_comp:
                                        num = levenshtein(p, p_comp)
                                        perc = 100 - num / len(p_comp) * 100
                                        if perc > 90:
                                            med_perc += perc
                                            tem_word = True
                                            break
                                    if not tem_word:
                                        subString = False
                                if subString:
                                    perc_max = med_perc / len(words)
                                    candidato = df_CIPE_norm.at[y, 'Termo']
                                    metodo = 51
                                    y_aux = y
                                    df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                    df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                    df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                    df_termos_match.at[zz, 'ICNPmod'] = candidato
                                    df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                    df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                    df_termos_match.at[zz, 'Regra'] = 51
                                    df_termos_match.at[zz, 'perc'] = perc_max
                                    zz += 1
                    if not (metodo == 51 or metodo == 61):
                        words = df_termos_lemma.at[x, 'Termos'].split()
                        if len(words) > 1:
                            for y in range(0, len(df_CIPE_norm)):
                                word_comp = df_CIPE_lemma.at[y, 'Termo'].split()
                                med_perc = 0
                                if len(words) < len(word_comp):
                                    subString = True
                                    for p in words:
                                        tem_word = False
                                        for p_comp in word_comp:
                                            num = levenshtein(p, p_comp)
                                            perc = 100 - num / len(p_comp) * 100
                                            if perc > 90:
                                                med_perc += perc
                                                tem_word = True
                                                break
                                        if not tem_word:
                                            subString = False
                                    if subString:
                                        perc_max = med_perc / len(words)
                                        candidato = df_CIPE_norm.at[y, 'Termo']
                                        metodo = 52
                                        y_aux = y
                                        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                        df_termos_match.at[zz, 'ICNPmod'] = candidato
                                        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                        df_termos_match.at[zz, 'Regra'] = 52
                                        df_termos_match.at[zz, 'perc'] = perc_max
                                        zz += 1
                                elif len(words) > len(word_comp):
                                    supString = True
                                    for p in word_comp:
                                        tem_word = False
                                        for p_comp in words:
                                            num = levenshtein(p, p_comp)
                                            perc = 100 - num / len(p_comp) * 100
                                            if perc > 90:
                                                med_perc += perc
                                                tem_word = True
                                                break
                                        if not tem_word:
                                            supString = False
                                    if supString:
                                        perc_max = med_perc / len(word_comp)
                                        candidato = df_CIPE_norm.at[y, 'Termo']
                                        metodo = 62
                                        y_aux = y
                                        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                        df_termos_match.at[zz, 'ICNPmod'] = candidato
                                        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                        df_termos_match.at[zz, 'Regra'] = 62
                                        df_termos_match.at[zz, 'perc'] = perc_max
                                        zz += 1
                        elif len(words) == 1:
                            for y in range(0, len(df_CIPE_norm)):
                                word_comp = df_CIPE_lemma.at[y, 'Termo'].split()
                                med_perc = 0
                                if len(words) < len(word_comp):
                                    subString = True
                                    for p in words:
                                        tem_word = False
                                        for p_comp in word_comp:
                                            num = levenshtein(p, p_comp)
                                            perc = 100 - num / len(p_comp) * 100
                                            if perc > 90:
                                                med_perc += perc
                                                tem_word = True
                                                break
                                        if not tem_word:
                                            subString = False
                                    if subString:
                                        perc_max = med_perc / len(words)
                                        candidato = df_CIPE_norm.at[y, 'Termo']
                                        metodo = 52
                                        y_aux = y
                                        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                        df_termos_match.at[zz, 'ICNPmod'] = candidato
                                        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                        df_termos_match.at[zz, 'Regra'] = 52
                                        df_termos_match.at[zz, 'perc'] = perc_max
                                        zz += 1
                        if not (metodo == 52 or metodo == 62):
                            words = df_termos_stem.at[x, 'Termos'].split()
                            if len(words) > 1:
                                for y in range(0, len(df_CIPE_norm)):
                                    word_comp = df_CIPE_stem.at[y, 'Termo'].split()
                                    med_perc = 0
                                    if len(words) < len(word_comp):
                                        subString = True
                                        for p in words:
                                            tem_word = False
                                            for p_comp in word_comp:
                                                num = levenshtein(p, p_comp)
                                                perc = 100 - num / len(p_comp) * 100
                                                if perc > 90:
                                                    med_perc += perc
                                                    tem_word = True
                                                    break
                                            if not tem_word:
                                                subString = False
                                        if subString:
                                            perc_max = med_perc / len(words)
                                            candidato = df_CIPE_norm.at[y, 'Termo']
                                            metodo = 53
                                            y_aux = y
                                            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                            df_termos_match.at[zz, 'ICNPmod'] = candidato
                                            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                            df_termos_match.at[zz, 'Regra'] = 53
                                            df_termos_match.at[zz, 'perc'] = perc_max
                                            zz += 1
                                    elif len(words) > len(word_comp):
                                        supString = True
                                        for p in word_comp:
                                            tem_word = False
                                            for p_comp in words:
                                                num = levenshtein(p, p_comp)
                                                perc = 100 - num / len(p_comp) * 100
                                                if perc > 90:
                                                    med_perc += perc
                                                    tem_word = True
                                                    break
                                            if not tem_word:
                                                supString = False
                                        if supString:
                                            perc_max = med_perc / len(word_comp)
                                            candidato = df_CIPE_norm.at[y, 'Termo']
                                            metodo = 63
                                            y_aux = y
                                            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                            df_termos_match.at[zz, 'ICNPmod'] = candidato
                                            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                            df_termos_match.at[zz, 'Regra'] = 63
                                            df_termos_match.at[zz, 'perc'] = perc_max
                                            zz += 1
                            elif len(words) == 1:
                                for y in range(0, len(df_CIPE_norm)):
                                    word_comp = df_CIPE_stem.at[y, 'Termo'].split()
                                    med_perc = 0
                                    if len(words) < len(word_comp):
                                        subString = True
                                        for p in words:
                                            tem_word = False
                                            for p_comp in word_comp:
                                                num = levenshtein(p, p_comp)
                                                perc = 100 - num / len(p_comp) * 100
                                                if perc > 90:
                                                    med_perc += perc
                                                    tem_word = True
                                                    break
                                            if not tem_word:
                                                subString = False
                                        if subString:
                                            perc_max = med_perc / len(words)
                                            candidato = df_CIPE_norm.at[y, 'Termo']
                                            metodo = 53
                                            y_aux = y
                                            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                            df_termos_match.at[zz, 'ICNPmod'] = candidato
                                            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                            df_termos_match.at[zz, 'Regra'] = 53
                                            df_termos_match.at[zz, 'perc'] = perc_max
                                            zz += 1
    '''if metodo == 1 and perc_max >= 90:
        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
        df_termos_match.at[zz, 'ICNPmod'] = candidato
        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
        df_termos_match.at[zz, 'Regra'] = 1
        df_termos_match.at[zz, 'perc'] = perc_max
        zz += 1
    elif metodo == 2 and perc_max >= 90:
        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
        df_termos_match.at[zz, 'ICNPmod'] = candidato
        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
        df_termos_match.at[zz, 'Regra'] = 2
        df_termos_match.at[zz, 'perc'] = perc_max
        zz += 1
    elif metodo == 3 and perc_max >= 90:
        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
        df_termos_match.at[zz, 'ICNPmod'] = candidato
        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
        df_termos_match.at[zz, 'Regra'] = 3
        df_termos_match.at[zz, 'perc'] = perc_max
        zz += 1
    elif not (metodo == 51 or metodo == 61 or metodo == 52 or metodo == 62 or metodo == 53 or metodo == 63
              or metodo == 12 or metodo == 22 or metodo == 32):
        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
        df_termos_match.at[zz, 'Regra'] = 0
        df_termos_match.at[zz, 'perc'] = 0
        df_termos_match.at[zz, 'ICNPcode'] = ''
        df_termos_match.at[zz, 'ICNPterm'] = ''
        df_termos_match.at[zz, 'ICNPVersao'] = ''
        df_termos_match.at[zz, 'ICNPeixo'] = ''
        df_termos_match.at[zz, 'numICNP'] = 0
        zz += 1
    else:
        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
        df_termos_match.at[zz, 'Regra'] = 0
        df_termos_match.at[zz, 'perc'] = 0
        df_termos_match.at[zz, 'ICNPcode'] = ''
        df_termos_match.at[zz, 'ICNPterm'] = ''
        df_termos_match.at[zz, 'ICNPVersao'] = ''
        df_termos_match.at[zz, 'ICNPeixo'] = ''
        df_termos_match.at[zz, 'numICNP'] = 0
        zz += 1'''
    if perc_max < 90:
        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
        df_termos_match.at[zz, 'Regra'] = 0
        df_termos_match.at[zz, 'perc'] = 0
        df_termos_match.at[zz, 'ICNPcode'] = ''
        df_termos_match.at[zz, 'ICNPterm'] = ''
        df_termos_match.at[zz, 'ICNPVersao'] = ''
        df_termos_match.at[zz, 'ICNPeixo'] = ''
        df_termos_match.at[zz, 'numICNP'] = 0
        zz += 1
    print(df_termos_norm.at[x, 'Termos'], ' tem o par ', candidato, ' com a porcentagem ', perc_max)
    perc_max = 0

83.33333333333334
abertura corporal  tem o par  abertura corporal  com a porcentagem  100.0
abrangencia  tem o par  abortamento  com a porcentagem  0
abrasao  tem o par  bandagem   com a porcentagem  0
absenteismo  tem o par  aument  com a porcentagem  0
absorver  tem o par  dispositivo absorver coletar  com a porcentagem  100.0
abstinencia  tem o par  sintoma abstinencia (de afastamento de retirada algo)  com a porcentagem  100.0
abuso alcool (ou alcoolismo)  tem o par  abuso alcool (ou alcoolismo)  com a porcentagem  100.0
abuso tabaco (ou fumo)  tem o par  abuso tabaco (ou fumo)  com a porcentagem  100.0
acao  tem o par  acao  com a porcentagem  100.0
aceitacao  tem o par  aceitacao  com a porcentagem  100.0
acessibilidade  tem o par  suscetibilidade  com a porcentagem  0
acesso  tem o par  acesso  com a porcentagem  100.0
acido  tem o par  aceit  com a porcentagem  0
acidose  tem o par  acidose respiratoria  com a porcentagem  100.0
adaptacao  tem o par  adaptacao  com a porcentage

casuistica coleta  tem o par  coletar  com a porcentagem  100.0
categorizar  tem o par  categorizar  com a porcentagem  100.0
causa  tem o par  motivar   com a porcentagem  0
cefalosporina  tem o par  regia corporal  com a porcentagem  0
celula  tem o par  celula cervical  com a porcentagem  100.0
choque  tem o par  choque  com a porcentagem  100.0
cicatrizacao ferida  tem o par  cicatrizacao ferida  com a porcentagem  100.0
ciencia  tem o par  conhecimento  com a porcentagem  0
cientifico  tem o par  identific  com a porcentagem  0
cirurgia  tem o par  cirurgia  com a porcentagem  100.0
citotoxicas  tem o par  cataton  com a porcentagem  0
claudicacao  tem o par  falar   com a porcentagem  0
clindamicina  tem o par  confin domicili  com a porcentagem  0
clinica  tem o par  clinica  com a porcentagem  100.0
clorexidina  tem o par  colabor medic  com a porcentagem  0
coagulase  tem o par  convulsa  com a porcentagem  0
cobertura  tem o par  dispositivo cobertura  com a porcentagem  100.

consulta acompanhamento (ou consulta subsequente)  tem o par  consulta acompanhamento (ou consulta subsequente)  com a porcentagem  100.0
['consulta', 'acompanhamento']
tempo sinonimos,  4.084670305252075
---------dicionarios------- consulta acompanhamento
{'consulta': ['parecer', 'conselho', 'opiniao', 'consulta'], 'acompanhamento': ['comitiva', 'seguimento', 'sequito', 'acompanhamento']}
{'consulta': ['parecer ', 'conselho ', 'opiniao ', 'consultar '], 'acompanhamento': ['comitiva ', 'seguir ', 'sequito ', 'acompanhar ']}
{'consulta': ['parec', 'conselh', 'opinia', 'consult'], 'acompanhamento': ['comit', 'seguiment', 'sequit', 'acompanh']}
consulta acompanhamento  numero de combinações:  16
tempo posib,  0.0
['parecer comitiva', 'parecer seguimento', 'parecer sequito', 'parecer acompanhamento', 'conselho comitiva', 'conselho seguimento', 'conselho sequito', 'conselho acompanhamento', 'opiniao comitiva', 'opiniao seguimento', 'opiniao sequito', 'opiniao acompanhamento', 'consulta comi

doppler  tem o par  complic  com a porcentagem  0
dor  tem o par  dor  com a porcentagem  100.0
dor por ferida  tem o par  dor por ferida  com a porcentagem  100.0
dormencia  tem o par  envelhecimento  com a porcentagem  0
drenagem  tem o par  tubo drenagem  com a porcentagem  100.0
droga  tem o par  droga  com a porcentagem  100.0
duracao  tem o par  duracao  com a porcentagem  100.0
eczema  tem o par  eczema  com a porcentagem  100.0
edema  tem o par  edema  com a porcentagem  100.0
efeito  tem o par  papel efeito colateral (ou secundario)  com a porcentagem  100.0
efeito analgesico  tem o par  efeito analgesico  com a porcentagem  100.0
efeito antibiotico  tem o par  efeito antibiotico  com a porcentagem  100.0
efeito colateral  tem o par  efeito colateral  com a porcentagem  100.0
eficiencia  tem o par  deficiencia imunologica  com a porcentagem  90.9090909090909
elastico  tem o par  meia elastica  com a porcentagem  100.0
elevar  tem o par  elevar  com a porcentagem  100.0
elogiar

gentamicina  tem o par  gerenc micca  com a porcentagem  0
glandula  tem o par  glandula  com a porcentagem  100.0
glicemia  tem o par  gerenciar hipoglicemia  com a porcentagem  100.0
globina  tem o par  alodin  com a porcentagem  0
glossario  tem o par  ver cavitario   com a porcentagem  0
grau  tem o par  grau  com a porcentagem  100.0
gravidade  tem o par  gravidade  com a porcentagem  100.0
grupo  tem o par  grupo  com a porcentagem  100.0
hemacia  tem o par  ver topico   com a porcentagem  0
hematologia  tem o par  hematom  com a porcentagem  0
hemoderivados  tem o par  hemoterap  com a porcentagem  0
hemorragia  tem o par  hemorragia  com a porcentagem  100.0
hidratacao  tem o par  terapia liquidos (ou hidratacao)  com a porcentagem  90.9090909090909
hidroterapia  tem o par  hidroterapia  com a porcentagem  100.0
hiperglicemia  tem o par  hiperglicemia  com a porcentagem  100.0
hiperpigmentacao  tem o par  hipervitaminos  com a porcentagem  0
hipersensibilidade  tem o par  sensi

lesao  tem o par  lesao  com a porcentagem  100.0
lesao  tem o par  lesao  com a porcentagem  100.0
letalidade  tem o par  fertilidade  com a porcentagem  0
leucocito  tem o par  insignific  com a porcentagem  0
leucocitose  tem o par  lencol movel  com a porcentagem  0
lidar  tem o par  lidar  com a porcentagem  100.0
ligacao afetiva  tem o par  ligacao afetiva  com a porcentagem  100.0
limiar  tem o par  inicio  com a porcentagem  0
lise  tem o par  ris  com a porcentagem  0
literatura  tem o par  biblioterapia   com a porcentagem  0
localizacao  tem o par  localizacao ferida  com a porcentagem  100.0
localizacao ferida  tem o par  localizacao ferida  com a porcentagem  100.0
maceracao  tem o par  maceracao  com a porcentagem  100.0
macro  tem o par  grande  com a porcentagem  0
macula  tem o par  inflam  com a porcentagem  0
mais  tem o par  molh  com a porcentagem  0
manejar  tem o par  capacidade manejar (controlar)o regime, positiva  com a porcentagem  100.0
manejo  tem o par  te

palavra chave  tem o par  ferir fechar   com a porcentagem  0
paliacao  tem o par  prover (proporcionar, fornecer) paliacao  com a porcentagem  100.0
palpar  tem o par  palpar  com a porcentagem  100.0
panorama  tem o par  visitar   com a porcentagem  0
papaina  tem o par  paralis  com a porcentagem  0
papel apoio  tem o par  papel apoio  com a porcentagem  100.0
papel estudante  tem o par  papel estudante  com a porcentagem  100.0
papel genero  tem o par  papel genero  com a porcentagem  100.0
papel lazer  tem o par  papel lazer  com a porcentagem  100.0
papel terapeuta ocupacional  tem o par  papel terapeuta ocupacional  com a porcentagem  100.0
papel trabalho  tem o par  papel trabalho  com a porcentagem  100.0
papel espiritual  tem o par  papel espiritual  com a porcentagem  100.0
patologia  tem o par  processo patologico  com a porcentagem  100.0
pe  tem o par  pe  com a porcentagem  100.0
pele  tem o par  pele  com a porcentagem  100.0
pelicula  tem o par  ritmar   com a porcenta

regiao corporal  tem o par  regiao corporal  com a porcentagem  100.0
regime medicamentoso  tem o par  regime medicamentoso  com a porcentagem  100.0
registrar  tem o par  registrar  com a porcentagem  100.0
registro  tem o par  raivar   com a porcentagem  0
regra  tem o par  lei  com a porcentagem  0
regular  tem o par  regular  com a porcentagem  100.0
relatar  tem o par  relatar  com a porcentagem  100.0
remover  tem o par  remover  com a porcentagem  100.0
repeticao  tem o par  desolacao   com a porcentagem  0
reposicao  tem o par  posicao supina (ou decubito dorsal)  com a porcentagem  100.0
resfriamento  tem o par  dispositivo aquecimento esfriamento  com a porcentagem  90.9090909090909
['resfriamento', 'súbito']
tempo sinonimos,  4.1608617305755615
---------dicionarios------- resfriamento subito
{'resfriamento': ['esfriamento', 'arrefecimento', 'resfriamento'], 'súbito': ['subitamente', 'imediato', 'inesperado', 'inopino', 'repentino', 'subitaneo', 'impeto', 'assalto', 'súbito']

servicos promocao saude  tem o par  saude  com a porcentagem  100.0
servidor  tem o par  esfregar   com a porcentagem  0
['sigla', 'pressure', 'ulcer', 'scale', 'for', 'healing']
tempo sinonimos,  12.1422119140625
---------dicionarios------- sigla  pressure ulcer scale for healing
{'sigla': ['sigla'], 'pressure': ['pressure'], 'ulcer': ['ulcer'], 'scale': ['scale'], 'for': ['moda', 'uso', 'costume', 'for'], 'healing': ['healing']}
{'sigla': ['siglar '], 'pressure': ['pressure '], 'ulcer': ['ulcer '], 'scale': ['scale '], 'for': ['modo ', 'usar ', 'costume ', 'for '], 'healing': ['healing ']}
{'sigla': ['sigl'], 'pressure': ['pressur'], 'ulcer': ['ulcer'], 'scale': ['scal'], 'for': ['mod', 'uso', 'costum', 'for'], 'healing': ['healing']}
sigla  pressure ulcer scale for healing  numero de combinações:  4
tempo posib,  0.0
['sigla pressure ulcer scale moda healing', 'sigla pressure ulcer scale uso healing', 'sigla pressure ulcer scale costume healing', 'sigla pressure ulcer scale for heal

triglicerol cadeia  tem o par  risc ulcer por pressa  com a porcentagem  0
troca gasosa  tem o par  troca gasosa  com a porcentagem  100.0
trocar cobertura ferida (ou curativo)  tem o par  trocar cobertura ferida (ou curativo)  com a porcentagem  100.0
trombose  tem o par  trombose venosa profunda, ausente  com a porcentagem  100.0
turvacao  tem o par  agitacao  com a porcentagem  0
ulcera  tem o par  ulcera  com a porcentagem  100.0
ulcera arterial  tem o par  ulcera arterial  com a porcentagem  100.0
ulcera por pressao  tem o par  ulcera por pressao  com a porcentagem  100.0
['úlcera', 'vascular']
tempo sinonimos,  5.050112009048462
---------dicionarios------- ulcera vascular
{'úlcera': ['chaga', 'úlcera'], 'vascular': ['vascular']}
{'úlcera': ['chagar ', 'úlcera '], 'vascular': ['vascular ']}
{'úlcera': ['chag', 'úlcer'], 'vascular': ['vascul']}
ulcera vascular  numero de combinações:  2
tempo posib,  0.0
['chaga vascular', 'úlcera vascular']
ulcera vascular  tem o par  ulcera  com 

In [36]:
writer = pd.ExcelWriter('CliniMap_CIPE_escol_sandra_2.xlsx')
df_termos_match.to_excel(writer, 'Match')
writer.save()

In [37]:
print(df_termos_norm['Termos'].size)
print(achou)
print((achou/df_termos_norm['Termos'].size)*100)

742
0
0.0


In [38]:
df_termos_match

,Termos,Regra,perc,ICNPcode,ICNPterm,ICNPmod,ICNPeixo,ICNPVersao,numICNP
0,abertura corporal,1,100,10003422,abertura corporal,abertura corporal,L,1,NaN
1,abrangência,0,0,,,NaN,,,0.0
2,abrasão,0,0,,,NaN,,,0.0
3,absenteísmo,0,0,,,NaN,,,0.0
4,absorver,12,100,10002641,aspirar,aspirar,A,1,NaN
5,absorver,51,100,10000289,dispositivo para absorver ou coletar,dispositivo absorver coletar,M,1,NaN
6,abstinência,51,100,10040085,controle do sintoma de abstinência (de afastam...,controle sintoma abstinencia (de afastamento d...,DE/RE,2013,NaN
7,abstinência,51,100,10039947,sintoma de abstinência (de afastamento ou de r...,sintoma abstinencia (de afastamento de retirad...,DE/RE,2013,NaN
8,abstinência,51,100,10038718,gerenciar sintoma de abstinência (de afastamen...,gerenciar sintoma abstinencia (de afastamento ...,IE,2013,NaN
9,abstinência,51,100,10036343,monitorar abstinência (afastamento ou retirada...,monitorar abstinencia (afastamento retirada algo),IE,2013,NaN


In [83]:
import re
import time
import pymysql
import pandas as pd
from translate_api.translate_api import api
from numpy import *
import unidecode
import snowballstemmer
from hunspell import Hunspell
import numpy as np
from dicio import Dicio

In [84]:
def removearticles(text):
    text = re.sub('"','',text)
    #text = re.sub('^(\s*)(de|do|dos|da|das|o|os|a|as|com|para|pra|e|ou|na|nas|no|nos|ao|aos|em|mg|ml)(\s+)','',text)
    return re.sub('((^\s*)|(\s+))(de|do|dos|da|das|o|os|a|as|com|para|pra|e|ou|na|nas|no|nos|ao|aos|em|mg|ml)(($\s*)|(\s+))', ' ', text)

def tem_num(s):
    return bool(re.match('.*\d.*', s))

def lematizador(text):
    auxt = str('')
    words = text.split()
    for x in words:
        if h.spell(x):
            if len(h.stem(x)) != 0:
                auxt += h.stem(x)[0] + ' '
            else:
                auxt += x + ' '
        else:
            auxt += x + ' '
    auxt.strip()
    return auxt

def stemmer(text):
    stemmer = snowballstemmer.stemmer('portuguese')
    return ' '.join(stemmer.stemWords(text.split())).strip()

def comb(lis, dict_lis):
    aux = []
    for s in lis:
        for ss in dict_lis:
            aux.append(s+' '+ss)
    return aux

In [85]:
h = Hunspell('pt_BR', hunspell_data_dir='pt_BR')

read = pd.ExcelFile('procedim.xlsx')
df_termos = read.parse('Planilha1')

cnx = pymysql.connect(user='root', password='1234',database='annotationtool')
cursor = cnx.cursor()
sqlstr = ("SELECT id FROM annotationtool.textos where not original_id like 'CAJURU%';")
cursor.execute(sqlstr)
ids = []
y = 0
for (id,) in cursor:
    ids.insert(y,id)
    y+=1
cursor.close()
cnx.close()

In [87]:
termos_compostos = pd.DataFrame(data=None, columns=['texto_id','Termos','termo_norm','tk_id','tag','termoComposto_id','abreviatura','umlsCUI'])
cnx = pymysql.connect(user='root', password='1234',database='annotationtool')
cursor = cnx.cursor()
z = 0
for id_aux in ids:
    if id_aux < 9930:
        print(id_aux)
        continue
    sqlstr = ("SELECT "
              "	a.*, "
              "	GROUP_CONCAT(token_id ORDER BY token_id ASC SEPARATOR ',') as token_ids, "
              "	GROUP_CONCAT(tk.token ORDER BY token_id ASC SEPARATOR ' ') as token_name, "
              "	tg.tag "
              "FROM "
              "	anotacoes a "
              "INNER JOIN tokens tk ON (a.token_id = tk.id) "
              "INNER JOIN sentencas s ON (tk.sentenca_id = s.id) "
              "INNER JOIN tags tg ON (a.tag_id = tg.id) "
              "WHERE "
              "	s.texto_id = %s AND a.adjudicador =  1 AND termocomposto_id IS NOT NULL "
              "GROUP BY "
              "	termocomposto_id, tag_id")
    cursor.execute(sqlstr, str(id_aux))
    token_ids_aux = []
    token_name_aux = []
    tag_aux = []
    termocomposto_id_aux = []
    abreviatura_aux = []
    umlscui_aux = []
    y = 0
    for (id, tag_id, token_id, anotador_id, termocomposto_id, status, adjudicador, 
         dataanotacao, abreviatura, umlscui, snomedctid, token_ids, token_name, tag) in cursor:
        token_ids_aux.insert(y,token_ids)
        token_name_aux.insert(y,token_name)
        tag_aux.insert(y,tag)
        termocomposto_id_aux.insert(y,termocomposto_id)
        abreviatura_aux.insert(y,abreviatura)
        umlscui_aux.insert(y,umlscui)
        y+=1
    for i in range(0,len(token_ids_aux)):
        if tem_num(token_name_aux[i]):
            continue
        term_norm = removearticles(unidecode.unidecode(str(token_name_aux[i]))).strip()
        locTerm = termos_compostos.loc[termos_compostos['termo_norm'] == term_norm]
        if locTerm['termo_norm'].empty:
            termos_compostos.at[z,'texto_id'] = id_aux
            termos_compostos.at[z,'Termos'] = token_name_aux[i]
            termos_compostos.at[z,'termo_norm'] = term_norm
            termos_compostos.at[z,'tk_id'] = token_ids_aux[i]
            termos_compostos.at[z,'tag'] = tag_aux[i]
            termos_compostos.at[z,'termoComposto_id'] = termocomposto_id_aux[i]
            termos_compostos.at[z,'abreviatura'] = abreviatura_aux[i]
            termos_compostos.at[z,'umlsCUI'] = umlscui_aux[i]
            z+=1
        else:
            if tag_aux[i] == "Abbreviation":
                termos_compostos.at[locTerm.index[0],'abreviatura'] = abreviatura_aux[i]
                termos_compostos.at[locTerm.index[0],'Termos'] = abreviatura_aux[i]
            if locTerm.at[locTerm.index[0],'termoComposto_id'] == termocomposto_id_aux[i]:
                tags = locTerm.at[locTerm.index[0],'tag'].split(';')
                tags_aux = tags
                tag_str = ''
                First = True
                for t in tags:
                    if t != tag_aux[i]:
                        tags_aux.append(tag_aux[i])
                for tg in tags_aux:
                    if First:
                        tag_str += tg
                        First = False
                    else:
                        tag_str += ';' + tg
                termos_compostos.at[locTerm.index[0],'tag'] = tag_str
cursor.close()
cnx.close()

9053
9128
9135
9160
9162
9165
9185
9186
9187
9193
9228
9229
9230
9231
9232
9233
9234
9235
9236
9237
9238
9239
9240
9241
9242
9243
9244
9245
9246
9247
9248
9249
9250
9251
9252
9253
9254
9255
9256
9257
9258
9259
9260
9262
9275
9320
9321
9322
9323
9324
9325
9326
9327
9328
9329
9330
9331
9332
9333
9334
9335
9336
9337
9338
9339
9340
9341
9342
9343
9344
9345
9346
9347
9348
9349
9350
9351
9352
9353
9354
9355
9356
9357
9358
9359
9369
9370
9371
9375
9378
9383
9384
9389
9390
9393
9394
9395
9397
9400
9401
9404
9405
9406
9408
9410
9411
9412
9413
9414
9415
9416
9417
9418
9419
9420
9421
9422
9424
9425
9426
9428
9430
9431
9432
9436
9437
9438
9439
9440
9441
9444
9445
9451
9453
9454
9455
9456
9457
9460
9461
9462
9463
9465
9466
9467
9468
9469
9472
9474
9475
9476
9477
9478
9481
9482
9484
9485
9486
9491
9492
9494
9496
9497
9498
9499
9500
9501
9502
9504
9506
9507
9508
9509
9510
9511
9512
9513
9514
9515
9516
9517
9518
9519
9520
9521
9522
9523
9524
9525
9526
9527
9528
9529
9530
9531
9532
9533
9534
9535
9536


In [88]:
termos_simples = pd.DataFrame(data=None, columns=['texto_id','Termos','termo_norm','tk_id','tag','termoComposto_id','abreviatura','umlsCUI'])
cnx = pymysql.connect(user='root', password='1234',database='annotationtool')
cursor = cnx.cursor()
cursor2 = cnx.cursor()
z = 0
for id_aux in ids:
    if int(id_aux) < 9930:
        continue
    sqlstr = ("SELECT  "
              "	a.*, tk.token, tg.tag  "
              "FROM  "
              "	anotacoes a  "
              "INNER JOIN tokens tk ON (a.token_id = tk.id)  "
              "INNER JOIN sentencas s ON (tk.sentenca_id = s.id)  "
              "INNER JOIN tags tg ON (a.tag_id = tg.id)  "
              "WHERE "  
              "	s.texto_id = %s AND a.adjudicador = 1 AND termocomposto_id IS NULL")
    cursor.execute(sqlstr, str(id_aux))
    token_id_aux = []
    token_name_aux = []
    tag_aux = []
    termocomposto_id_aux = []
    abreviatura_aux = []
    umlscui_aux = []
    y = 0
    for (id, tag_id, token_id, anotador_id, termocomposto_id, status, adjudicador, 
         dataanotacao, abreviatura, umlscui, snomedctid, token, tag) in cursor:
        token_id_aux.insert(y,token_id)
        token_name_aux.insert(y,token)
        tag_aux.insert(y,tag)
        termocomposto_id_aux.insert(y,termocomposto_id)
        abreviatura_aux.insert(y,abreviatura)
        umlscui_aux.insert(y,umlscui)
        y+=1
    for i in range(0,len(token_id_aux)):
        if tem_num(token_name_aux[i]):
            continue
        term_norm = removearticles(unidecode.unidecode(str(token_name_aux[i]))).strip()
        locTerm = termos_simples.loc[termos_simples['termo_norm'] == term_norm]
        if locTerm['termo_norm'].empty:
            termos_simples.at[z,'texto_id'] = id_aux
            termos_simples.at[z,'Termos'] = token_name_aux[i]
            termos_simples.at[z,'termo_norm'] = term_norm
            termos_simples.at[z,'tk_id'] = token_id_aux[i]
            termos_simples.at[z,'tag'] = tag_aux[i]
            termos_simples.at[z,'termoComposto_id'] = termocomposto_id_aux[i]
            termos_simples.at[z,'abreviatura'] = abreviatura_aux[i]
            termos_simples.at[z,'umlsCUI'] = umlscui_aux[i]
            z+=1
        else:
            if tag_aux[i] == "Abbreviation":
                termos_simples.at[locTerm.index[0],'abreviatura'] = abreviatura_aux[i]
                if termos_simples.at[locTerm.index[0],'abreviatura'] != None:
                    termos_simples.at[locTerm.index[0],'Termos'] = abreviatura_aux[i]
                else:
                    sqlstr2 = ("SELECT acronimo,expansao FROM annotationtool.acronimos where acronimo=%s;")
                    cursor2.execute(sqlstr2, str(termos_simples.at[locTerm.index[0],'Termos']))
                    for (acronimo,expansao) in cursor2:
                        termos_simples.at[locTerm.index[0],'abreviatura'] = expansao
                        termos_simples.at[locTerm.index[0],'Termos'] = expansao
                        break
                    print('termo: ',termos_simples.at[locTerm.index[0],'Termos'])
                    print('abreviatura: ',termos_simples.at[locTerm.index[0],'abreviatura'])
            if locTerm.at[locTerm.index[0],'tk_id'] == token_id_aux[i]:
                tags = locTerm.at[locTerm.index[0],'tag'].split(';')
                tags_aux = tags
                tag_str = ''
                First = True
                for t in tags:
                    if t != tag_aux[i]:
                        tags_aux.append(tag_aux[i])
                for tg in tags_aux:
                    if First:
                        tag_str += tg
                        First = False
                    else:
                        tag_str += ';' + tg
                termos_simples.at[locTerm.index[0],'tag'] = tag_str
cursor2.close()
cursor.close()
cnx.close()

termo:  AO
abreviatura:  None
termo:  BARIÁTRICA
abreviatura:  None
termo:  BARIÁTRICA
abreviatura:  None
termo:  BARIÁTRICA
abreviatura:  None
termo:  PCTE
abreviatura:  None
termo:  exame físico
abreviatura:  exame físico
termo:  PCTE
abreviatura:  None
termo:  QP
abreviatura:  None
termo:  paciente
abreviatura:  paciente
termo:  exame físico
abreviatura:  None
termo:  escala de expressões faciais
abreviatura:  escala de expressões faciais
termo:  coronária direita
abreviatura:  coronária direita


# CLINIMAP

In [89]:
import re
import time
import pymysql
import pandas as pd
from translate_api.translate_api import api
from numpy import *
import unidecode
import snowballstemmer
from hunspell import Hunspell
import numpy as np
from dicio import Dicio


def levenshtein(source, target):
    if len(source) < len(target):
        return levenshtein(target, source)

    # So now we have len(source) >= len(target).
    if len(target) == 0:
        return len(source)

    # We call tuple() to force strings to be used as sequences
    # ('c', 'a', 't', 's') - numpy uses them as values by default.
    source = np.array(tuple(source))
    target = np.array(tuple(target))

    # We use a dynamic programming algorithm, but with the
    # added optimization that we only need the last two rows
    # of the matrix.
    previous_row = np.arange(target.size + 1)
    for s in source:
        # Insertion (target grows longer than source):
        current_row = previous_row + 1

        # Substitution or matching:
        # Target and source items are aligned, and either
        # are different (cost of 1), or are the same (cost of 0).
        current_row[1:] = np.minimum(
                current_row[1:],
                np.add(previous_row[:-1], target != s))

        # Deletion (target grows shorter than source):
        current_row[1:] = np.minimum(
                current_row[1:],
                current_row[0:-1] + 1)

        previous_row = current_row

    return previous_row[-1]

def removearticles(text):
    text = re.sub('"','',text)
    #text = re.sub('^(\s*)(de|do|dos|da|das|o|os|a|as|com|para|pra|e|ou|na|nas|no|nos|ao|aos|em|mg|ml)(\s+)','',text)
    return re.sub('((^\s*)|(\s+))(de|do|dos|da|das|o|os|a|as|com|para|pra|e|ou|na|nas|no|nos|ao|aos|em|mg|ml)(($\s*)|(\s+))', ' ', text)

def lematizador(text):
    auxt = str('')
    words = text.split()
    for x in words:
        if h.spell(x):
            if len(h.stem(x)) != 0:
                auxt += h.stem(x)[0] + ' '
            else:
                auxt += x + ' '
        else:
            auxt += x + ' '
    auxt.strip()
    return auxt

def stemmer(text):
    stemmer = snowballstemmer.stemmer('portuguese')
    return ' '.join(stemmer.stemWords(text.split())).strip()

def comb(lis, dict_lis):
    aux = []
    for s in lis:
        for ss in dict_lis:
            aux.append(s+' '+ss)
    return aux



ini = time.time()
h = Hunspell('pt_BR', hunspell_data_dir='pt_BR')

df_termos = pd.DataFrame(data=None, columns=['Termos'])
for x in range(0,len(termos_compostos['Termos'])):
    df_termos.at[x,'Termos'] = termos_compostos.at[x,'Termos']

cnx = pymysql.connect(user='root', password='1234',database='umls2017aa')
cursor = cnx.cursor()
sqlstr = ("SELECT  CUI, STR FROM umls2017aa.mrconso where LAT='POR' order by CUI;")
cursor.execute(sqlstr)
umls = []
CUI1 = []
STR1 = []
y = 0
for (CUI,STR) in cursor:
    CUI1.insert(y,CUI)
    STR1.insert(y,STR)
    y+=1
umls = array([STR1,CUI1])
umls2 = array([STR1,STR1,STR1,CUI1])
df_umls = pd.DataFrame(umls.transpose(),columns=['STR','CUI'])
df_umls_norm = pd.DataFrame(umls2.transpose(),columns=['STR', 'STRlemma', 'STRstem', 'CUI'])

sqlstr = ("SELECT  CUI, WD FROM umls2017aa.mrxw_por;")
cursor.execute(sqlstr)
umls = []
CUI2 = []
WD2 = []
y = 0
for (CUI,WD) in cursor:
    CUI2.insert(y,CUI)
    WD2.insert(y,WD)
    y+=1
umls_word = array([WD2,CUI2])
umls2_word = array([WD2,WD2,WD2,CUI2])
df_umls_word = pd.DataFrame(umls_word.transpose(),columns=['WD','CUI'])
df_umls_word_norm = pd.DataFrame(umls2_word.transpose(),columns=['WD', 'WDlemma', 'WDstem', 'CUI'])

cursor.close()
cnx.close()

for x in range(0,df_termos.size):
    df_termos.at[x,'Termos'] = str(df_termos.at[x,'Termos']).lower()

for x in range(0,df_umls['STR'].size):
    df_umls.at[x,'STR'] = str(df_umls.at[x,'STR']).lower()

for x in range(0,df_umls_word['WD'].size):
    df_umls_word.at[x,'WD'] = str(df_umls_word.at[x,'WD']).lower()

for x in range(0,df_umls_norm['STR'].size):
    df_umls_norm.at[x,'STR'] = str(df_umls_norm.at[x,'STR']).lower()
    df_umls_norm.at[x,'STRlemma'] = str(df_umls_norm.at[x,'STRlemma']).lower()
    df_umls_norm.at[x,'STRstem'] = str(df_umls_norm.at[x,'STRstem']).lower()

for x in range(0,df_umls_word_norm['WD'].size):
    df_umls_word_norm.at[x,'WD'] = str(df_umls_word_norm.at[x,'WD']).lower()
    df_umls_word_norm.at[x,'WDlemma'] = str(df_umls_word_norm.at[x,'WDlemma']).lower()
    df_umls_word_norm.at[x,'WDstem'] = str(df_umls_word_norm.at[x,'WDstem']).lower()

df_termos_norm = pd.DataFrame(data=None, columns=['Termos'])
df_termos_lemma = pd.DataFrame(data=None, columns=['Termos'])
df_termos_stem = pd.DataFrame(data=None, columns=['Termos'])

for x in range(0,df_termos.size):
    df_termos_norm.at[x,'Termos'] = removearticles(unidecode.unidecode(str(df_termos.at[x,'Termos']))).strip()

for x in range(0,df_termos.size):
    df_termos_lemma.at[x,'Termos'] = removearticles(unidecode.unidecode(lematizador(str(df_termos.at[x,'Termos']))))

for x in range(0,df_termos.size):
    df_termos_stem.at[x,'Termos'] = removearticles(unidecode.unidecode(stemmer(str(df_termos.at[x,'Termos']))))

for x in range(0,df_umls_norm['STR'].size):
    df_umls_norm.at[x,'STR'] = removearticles(unidecode.unidecode(str(df_umls_norm.at[x,'STR']))).strip()

for x in range(0,df_umls_norm['STRlemma'].size):
    df_umls_norm.at[x,'STRlemma'] = removearticles(unidecode.unidecode(lematizador(str(df_umls_norm.at[x,'STRlemma']))))

for x in range(0,df_umls_norm['STRstem'].size):
    df_umls_norm.at[x,'STRstem'] = removearticles(unidecode.unidecode(stemmer(str(df_umls_norm.at[x,'STRstem']))))

for x in range(0,df_umls_word_norm['WD'].size):
    df_umls_word_norm.at[x,'WD'] = unidecode.unidecode(str(df_umls_word_norm.at[x,'WD'])).strip()

for x in range(0,df_umls_word_norm['WDlemma'].size):
    df_umls_word_norm.at[x,'WDlemma'] = unidecode.unidecode(lematizador(str(df_umls_word_norm.at[x,'WDlemma'])))

for x in range(0,df_umls_word_norm['WDstem'].size):
    df_umls_word_norm.at[x,'WDstem'] = unidecode.unidecode(stemmer(str(df_umls_word_norm.at[x,'WDstem'])))

#iniR = time.time()
df_termos_match = pd.DataFrame(data=None, columns=['Termos', 'CUI', 'STR_UMLS', 'termo_match', 'Regra', 'num_lev'])

print(100-1/len('lucass')*100)

zz = 0
achou = 0
candidato = ''
perc_max = 0
metodo = 0
y_aux = 0

dicio = Dicio()

83.33333333333334


In [90]:
df_termos

,Termos
0,serviço particular
1,pupila fotorreagente
2,none
3,drusas perimaculares
4,atrofia de epitélio pigmentar retiniano
5,none
6,none
7,none
8,córnea clara
9,hipertensão ocular


In [99]:
df_termos_match = pd.DataFrame(data=None, columns=['Termos', 'CUI', 'STR_UMLS', 'termo_match', 'Regra', 'num_lev'])

In [100]:
'string'[0]

's'

In [102]:
cnx = pymysql.connect(user='root', password='1234',database='umls2017aa')
cursor = cnx.cursor()
for x in range(0,len(df_termos_norm)):
    num_max = len(df_termos_norm.at[x,'Termos'])+len(df_termos_norm.at[x,'Termos'])*0.25
    num_min = len(df_termos_norm.at[x,'Termos'])-len(df_termos_norm.at[x,'Termos'])*0.25
    print(df_termos_norm.at[x,'Termos'])
    print(num_perc)
    candidatos = pd.DataFrame(data=None, columns=['CUI', 'metodo', 'porc', 'Termos', 'STR_UMLS', 'termo_match'])
    n_candidato = 0
    words = df_termos_norm.at[x,'Termos'].split()
    words_lemma = df_termos_lemma.at[x,'Termos'].split()
    words_steam = df_termos_stem.at[x, 'Termos'].split()
    termo_ingles_norm = api(df_termos_norm.at[x,'Termos'], 'pt', 'en')
    print('o termo em ingles é: ', termo_ingles_norm)
    termo_ingles = api(df_termos.at[x,'Termos'], 'pt', 'en')
    print('o termo em ingles sem normalização é: ', termo_ingles)
    
    iniR = time.time()
    sqlstr = ("SELECT  CUI, STR FROM umls2017aa.mrconso where STR= %s and LAT='ENG' order by CUI;")
    cursor.execute(sqlstr, termo_ingles_norm)
    CUIS_aux = []
    for (CUI,STR) in cursor:
        if not CUI in CUIS_aux:
            CUIS_aux.append(CUI)
            candidatos.at[n_candidato,'CUI'] = CUI
            candidatos.at[n_candidato,'Termos'] = df_termos.at[x,'Termos']
            candidatos.at[n_candidato,'STR_UMLS'] = STR
            candidatos.at[n_candidato,'termo_match'] = df_termos_norm.at[x,'Termos']
            candidatos.at[n_candidato, 'metodo'] = 4
            candidatos.at[n_candidato, 'porc'] = 100
            n_candidato += 1
    
    sqlstr = ("SELECT  CUI, STR FROM umls2017aa.mrconso where STR= %s and LAT='ENG' order by CUI;")
    cursor.execute(sqlstr, termo_ingles)
    for (CUI,STR) in cursor:
        if not CUI in CUIS_aux:
            CUIS_aux.append(CUI)
            candidatos.at[n_candidato,'CUI'] = CUI
            candidatos.at[n_candidato,'Termos'] = df_termos.at[x,'Termos']
            candidatos.at[n_candidato,'STR_UMLS'] = STR
            candidatos.at[n_candidato,'termo_match'] = df_termos_norm.at[x,'Termos']
            candidatos.at[n_candidato, 'metodo'] = 4
            candidatos.at[n_candidato, 'porc'] = 100
            n_candidato += 1
    print(candidatos)
    achou_Term = False
    fimR = time.time()
    print('tempo, ', (fimR-iniR))
    perc_max = 0
cursor.close()
cnx.close()

servico particular
22.5
o termo em ingles é:  private service
o termo em ingles sem normalização é:  private service
Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
aqui  servico particular  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  35.17680382728577
pupila fotorreagente
22.5
o termo em ingles é:  photoreceptor pupil
o termo em ingles sem normalização é:  photoreceptor pupil
Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
aqui  pupila fotorreagente  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  41.96259045600891
none
22.5
o termo em ingles é:  none
o termo em ingles sem normalização é:  none
         CUI metodo porc Termos STR_UMLS termo_match
0   C0456148      4  100   none     None        none
1   C0549184      4  100   none     None        none
2   C1546509      4  100   none     none        none
3  

o termo em ingles é:  none
o termo em ingles sem normalização é:  none
         CUI metodo porc Termos STR_UMLS termo_match
0   C0456148      4  100   none     None        none
1   C0549184      4  100   none     None        none
2   C1546509      4  100   none     none        none
3   C1547191      4  100   none     none        none
4   C1550083      4  100   none     None        none
5   C1550437      4  100   none     None        none
6   C1551387      4  100   none     None        none
7   C1553523      4  100   none     none        none
8   C1556146      4  100   none     None        none
9   C1556147      4  100   none     None        none
10  C1556148      4  100   none     None        none
11  C1556150      4  100   none     None        none
12  C1556151      4  100   none     None        none
13  C1556152      4  100   none     None        none
14  C1706277      4  100   none     None        none
15  C4050154      4  100   none     None        none
16  C4050155      4  100   n

aqui  none  ---           CUI metodo porc Termos STR_UMLS termo_match
0   C0456148      4  100   none     None        none
14  C1706277      4  100   none     None        none
13  C1556152      4  100   none     None        none
12  C1556151      4  100   none     None        none
11  C1556150      4  100   none     None        none
10  C1556148      4  100   none     None        none
9   C1556147      4  100   none     None        none
15  C4050154      4  100   none     None        none
8   C1556146      4  100   none     None        none
6   C1551387      4  100   none     None        none
5   C1550437      4  100   none     None        none
4   C1550083      4  100   none     None        none
3   C1547191      4  100   none     none        none
2   C1546509      4  100   none     none        none
1   C0549184      4  100   none     None        none
7   C1553523      4  100   none     none        none
16  C4050155      4  100   none     None        none
tempo,  6.126916170120239
mas

aqui  dores articulares  ---          CUI metodo porc             Termos    STR_UMLS        termo_match
0  C0003862      4  100  dores articulares  Joint Pain  dores articulares
1  C1963066      4  100  dores articulares  Joint pain  dores articulares
2  C4085641      4  100  dores articulares  Joint Pain  dores articulares
tempo,  32.7315936088562
lesoes hiperemiada
22.5
o termo em ingles é:  hyperemiotic lesions
o termo em ingles sem normalização é:  hyperemiotic lesions
Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
aqui  lesoes hiperemiada  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  36.039159059524536
mao direita
22.5
o termo em ingles é:  right hand
o termo em ingles sem normalização é:  right hand
        CUI metodo porc       Termos    STR_UMLS  termo_match
0  C0230370      4  100  mão direita  Right hand  mao direita
1  C1288948      4  100  mão direita  Right hand  mao direita
aqui  

o termo em ingles é:  headache
o termo em ingles sem normalização é:  Headache
        CUI metodo porc         Termos  STR_UMLS termo_match
0  C0018681      4  100  dor de cabeça  Headache  dor cabeca
aqui  dor cabeca  ---          CUI metodo porc         Termos  STR_UMLS termo_match
0  C0018681      4  100  dor de cabeça  Headache  dor cabeca
tempo,  11.023988246917725
palpitacoes inicio subito desaparecimento gradual
22.5
o termo em ingles é:  palpitacoes beginning subito gradual disappearance
o termo em ingles sem normalização é:  sudden onset palpitations and gradual disappearance
Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
aqui  palpitacoes inicio subito desaparecimento gradual  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  67.41471791267395
dor toracica
22.5
o termo em ingles é:  Chest pain
o termo em ingles sem normalização é:  Chest pain
        CUI metodo porc        Termos    STR_UM

aqui  refrigerantes cola  ---          CUI metodo porc                 Termos     STR_UMLS  \
0  C0452444      4  100  refrigerantes de cola  cola drinks   

          termo_match  
0  refrigerantes cola  
tempo,  36.2611448764801
carcinoma garganta
22.5
o termo em ingles é:  throat carcinoma
o termo em ingles sem normalização é:  throat carcinoma
        CUI metodo porc                 Termos          STR_UMLS  \
0  C0740339      4  100  carcinoma de garganta  Throat Carcinoma   

          termo_match  
0  carcinoma garganta  
aqui  carcinoma garganta  ---          CUI metodo porc                 Termos          STR_UMLS  \
0  C0740339      4  100  carcinoma de garganta  Throat Carcinoma   

          termo_match  
0  carcinoma garganta  
tempo,  33.85846209526062
trauma ocular
22.5
o termo em ingles é:  ocular trauma
o termo em ingles sem normalização é:  ocular trauma
Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
aqui  trauma ocular  ---  Emp

o termo em ingles é:  clinical condition
o termo em ingles sem normalização é:  clinical condition
Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
aqui  quadro clinico  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  20.793199062347412


In [104]:
writer = pd.ExcelWriter('CliniMap_M4.xlsx')
df_termos_match.to_excel(writer, 'Match')
writer.save()


In [115]:
read = pd.ExcelFile('CliniMap_M4.xlsx')
clinMap_4 = read.parse('Match')

In [118]:
df_termos_match = pd.DataFrame(data=None, columns=['Termos', 'CUI', 'STR_UMLS', 'termo_match', 'Regra', 'num_lev'])

In [119]:
for x in range(0,len(df_termos_norm)):
    if df_termos_norm.at[x,'Termos'] == None:
        continue
    num_max = len(df_termos_norm.at[x,'Termos'])+len(df_termos_norm.at[x,'Termos'])*0.25
    num_min = len(df_termos_norm.at[x,'Termos'])-len(df_termos_norm.at[x,'Termos'])*0.25
    print(df_termos_norm.at[x,'Termos'])
    print(num_max, ' e ', num_min)
    candidatos = pd.DataFrame(data=None, columns=['CUI', 'metodo', 'porc', 'Termos', 'STR_UMLS', 'termo_match'])
    n_candidato = 0
    
    iniR = time.time()
    for y in range(0,len(df_umls_norm['STR'])):
        if df_umls_norm.at[y,'STR'] == None:
            continue
        if len(df_umls_norm.at[y,'STR']) < int(num_min):
            continue
        if len(df_umls_norm.at[y,'STR']) > int(num_max):
            continue
        if not df_umls_norm.at[y,'STR'][0] == df_termos_norm.at[x,'Termos'][0]:
            continue
        num = levenshtein(df_termos_norm.at[x,'Termos'],df_umls_norm.at[y,'STR'])
        if len(df_umls_norm.at[y,'STR']) > 0:
            perc = 100-num/len(df_umls_norm.at[y,'STR'])*100
        else:
            perc = 0
        if perc == 100:
            achou_Term = True
            candidatos.at[n_candidato,'CUI'] = df_umls_norm.at[y, 'CUI']
            candidatos.at[n_candidato,'Termos'] = df_termos.at[x,'Termos']
            candidatos.at[n_candidato,'STR_UMLS'] = df_umls_norm.at[y, 'STR']
            candidatos.at[n_candidato,'termo_match'] = df_termos_norm.at[x,'Termos']
            candidatos.at[n_candidato, 'metodo'] = 1
            candidatos.at[n_candidato, 'porc'] = perc
            n_candidato += 1
            break
        if perc >= 90:
            candidatos.at[n_candidato,'CUI'] = df_umls_norm.at[y, 'CUI']
            candidatos.at[n_candidato,'Termos'] = df_termos.at[x,'Termos']
            candidatos.at[n_candidato,'STR_UMLS'] = df_umls_norm.at[y, 'STR']
            candidatos.at[n_candidato,'termo_match'] = df_termos_norm.at[x,'Termos']
            candidatos.at[n_candidato, 'metodo'] = 1
            candidatos.at[n_candidato, 'porc'] = perc
            n_candidato += 1
    candidatos = candidatos.sort_values('metodo')
    print('aqui ', df_termos_norm.at[x,'Termos'] , ' --- ', candidatos)
    if len(candidatos['CUI']) == 0:
        df_termos_match.at[zz,'Termos'] = df_termos.at[x,'Termos']
        df_termos_match.at[zz,'Regra'] = 0
        df_termos_match.at[zz,'num_lev'] = 0
        df_termos_match.at[zz, 'CUI'] = ''
        df_termos_match.at[zz, 'STR_UMLS'] = ''
        df_termos_match.at[zz, 'termo_match'] = ''
        zz += 1
    else:
        for y in range(0, len(candidatos['CUI'])):
            df_termos_match.at[zz,'Termos'] = df_termos.at[x,'Termos']
            df_termos_match.at[zz,'Regra'] = candidatos.at[y,'metodo']
            df_termos_match.at[zz,'num_lev'] = candidatos.at[y,'porc']
            df_termos_match.at[zz, 'CUI'] = candidatos.at[y,'CUI']
            df_termos_match.at[zz, 'STR_UMLS'] = candidatos.at[y,'STR_UMLS']
            df_termos_match.at[zz, 'termo_match'] = candidatos.at[y,'termo_match']
            zz += 1
    fimR = time.time()
    print('tempo, ', (fimR-iniR))
    perc_max = 0
cursor.close()
cnx.close()

servico particular
22.5  e  13.5
aqui  servico particular  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  11.811729907989502
pupila fotorreagente
25.0  e  15.0
aqui  pupila fotorreagente  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  13.089941263198853
none
5.0  e  3.0
aqui  none  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  8.309885025024414
drusas perimaculares
25.0  e  15.0
aqui  drusas perimaculares  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  12.323413848876953
atrofia epitelio pigmentar retiniano
45.0  e  27.0
aqui  atrofia epitelio pigmentar retiniano  ---          CUI metodo     porc                                   Termos  \
0  C1720251      1  90.9091  atrofia de epitélio pigmentar retiniano   

                            STR_UMLS                           termo_m

aqui  none  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  8.348731756210327
furcula esternal
20.0  e  12.0
aqui  furcula esternal  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  10.384435892105103
limitacao articular
23.75  e  14.25
aqui  limitacao articular  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  10.75253415107727
medicamentos imunossupressores
37.5  e  22.5
aqui  medicamentos imunossupressores  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  11.514661073684692
cervical anterior-esquerda , posterior
47.5  e  28.5
aqui  cervical anterior-esquerda , posterior  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  16.04199719429016
fluxo medio
13.75  e  8.25
aqui  fluxo medio  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termo

In [120]:
writer = pd.ExcelWriter('CliniMap_M1.xlsx')
df_termos_match.to_excel(writer, 'Match')
writer.save()

In [121]:
read = pd.ExcelFile('CliniMap_M1.xlsx')
clinMap_1 = read.parse('Match')

In [132]:
df_termos_match = pd.DataFrame(data=None, columns=['Termos', 'CUI', 'STR_UMLS', 'termo_match', 'Regra', 'num_lev'])

In [133]:
for x in range(0,len(df_termos_lemma)):
    if df_termos_lemma.at[x,'Termos'] == None:
        continue
    
    num_max = len(df_termos_lemma.at[x,'Termos'])+len(df_termos_lemma.at[x,'Termos'])*0.25
    num_min = len(df_termos_lemma.at[x,'Termos'])-len(df_termos_lemma.at[x,'Termos'])*0.25
    print(df_termos_lemma.at[x,'Termos'])
    print(num_max, ' e ', num_min)
    candidatos = pd.DataFrame(data=None, columns=['CUI', 'metodo', 'porc', 'Termos', 'STR_UMLS', 'termo_match'])
    n_candidato = 0
    
    iniR = time.time()
    for y in range(0,len(df_umls_norm['STR'])):
        if df_umls_norm.at[y,'STRlemma'] == None:
            continue
        if len(df_umls_norm.at[y,'STRlemma']) < int(num_min):
            continue
        if len(df_umls_norm.at[y,'STRlemma']) > int(num_max):
            continue
        if not df_umls_norm.at[y,'STRlemma'][0] == df_termos_lemma.at[x,'Termos'][0]:
            continue
        num = levenshtein(df_termos_lemma.at[x,'Termos'],df_umls_norm.at[y,'STRlemma'])
        if len(df_umls_norm.at[y,'STRlemma']) > 0:
            perc = 100-num/len(df_umls_norm.at[y,'STRlemma'])*100
        else:
            perc = 0
        if perc == 100:
            achou_Term = True
            candidatos.at[n_candidato,'CUI'] = df_umls_norm.at[y, 'CUI']
            candidatos.at[n_candidato,'Termos'] = df_termos.at[x,'Termos']
            candidatos.at[n_candidato,'STR_UMLS'] = df_umls_norm.at[y, 'STR']
            candidatos.at[n_candidato,'termo_match'] = df_termos_lemma.at[x,'Termos']
            candidatos.at[n_candidato, 'metodo'] = 2
            candidatos.at[n_candidato, 'porc'] = perc
            n_candidato += 1
            break
        if perc >= 90:
            candidatos.at[n_candidato,'CUI'] = df_umls_norm.at[y, 'CUI']
            candidatos.at[n_candidato,'Termos'] = df_termos.at[x,'Termos']
            candidatos.at[n_candidato,'STR_UMLS'] = df_umls_norm.at[y, 'STR']
            candidatos.at[n_candidato,'termo_match'] = df_termos_lemma.at[x,'Termos']
            candidatos.at[n_candidato, 'metodo'] = 2
            candidatos.at[n_candidato, 'porc'] = perc
            n_candidato += 1
    candidatos = candidatos.sort_values('metodo')
    print('aqui ', df_termos_lemma.at[x,'Termos'] , ' --- ', candidatos)
    if len(candidatos['CUI']) == 0:
        df_termos_match.at[zz,'Termos'] = df_termos.at[x,'Termos']
        df_termos_match.at[zz,'Regra'] = 0
        df_termos_match.at[zz,'num_lev'] = 0
        df_termos_match.at[zz, 'CUI'] = ''
        df_termos_match.at[zz, 'STR_UMLS'] = ''
        df_termos_match.at[zz, 'termo_match'] = ''
        zz += 1
    else:
        for y in range(0, len(candidatos['CUI'])):
            df_termos_match.at[zz,'Termos'] = df_termos.at[x,'Termos']
            df_termos_match.at[zz,'Regra'] = candidatos.at[y,'metodo']
            df_termos_match.at[zz,'num_lev'] = candidatos.at[y,'porc']
            df_termos_match.at[zz, 'CUI'] = candidatos.at[y,'CUI']
            df_termos_match.at[zz, 'STR_UMLS'] = candidatos.at[y,'STR_UMLS']
            df_termos_match.at[zz, 'termo_match'] = candidatos.at[y,'termo_match']
            zz += 1
    fimR = time.time()
    print('tempo, ', (fimR-iniR))
    perc_max = 0

servico particular 
23.75  e  14.25
aqui  servico particular   ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  12.60045337677002
pupilar fotorreagente 
27.5  e  16.5
aqui  pupilar fotorreagente   ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  15.887298345565796
none 
6.25  e  3.75
aqui  none   ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  8.568651914596558
druso perimaculares 
25.0  e  15.0
aqui  druso perimaculares   ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  12.305294036865234
atrofiar epitelio pigmentar retiniano 
47.5  e  28.5
aqui  atrofiar epitelio pigmentar retiniano   ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  13.711738586425781
none 
6.25  e  3.75
aqui  none   ---  Empty DataFrame
Columns: [CUI, metodo, porc, Term

aqui  none   ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  10.110544919967651
furcula esternal 
21.25  e  12.75
aqui  furcula esternal   ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  11.8753981590271
limitacao articular 
25.0  e  15.0
aqui  limitacao articular   ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  11.817301750183105
medicar imunossupressor 
30.0  e  18.0
aqui  medicar imunossupressor   ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  13.862738132476807
cervical anterior-esquerda , posterior 
48.75  e  29.25
aqui  cervical anterior-esquerda , posterior   ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  17.60726571083069
fluxo medio 
15.0  e  9.0
aqui  fluxo medio   ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, 

In [134]:
writer = pd.ExcelWriter('CliniMap_M2.xlsx')
df_termos_match.to_excel(writer, 'Match')
writer.save()

In [135]:
read = pd.ExcelFile('CliniMap_M2.xlsx')
clinMap_2 = read.parse('Match')

In [136]:
df_termos_match = pd.DataFrame(data=None, columns=['Termos', 'CUI', 'STR_UMLS', 'termo_match', 'Regra', 'num_lev'])

In [137]:
for x in range(0,len(df_termos_stem)):
    if df_termos_stem.at[x,'Termos'] == None:
        continue
    
    num_max = len(df_termos_stem.at[x,'Termos'])+len(df_termos_stem.at[x,'Termos'])*0.25
    num_min = len(df_termos_stem.at[x,'Termos'])-len(df_termos_stem.at[x,'Termos'])*0.25
    print(df_termos_stem.at[x,'Termos'])
    print(num_max, ' e ', num_min)
    candidatos = pd.DataFrame(data=None, columns=['CUI', 'metodo', 'porc', 'Termos', 'STR_UMLS', 'termo_match'])
    n_candidato = 0
    
    iniR = time.time()
    for y in range(0,len(df_umls_norm['STR'])):
        if df_umls_norm.at[y,'STRstem'] == None:
            continue
        if len(df_umls_norm.at[y,'STRstem']) < int(num_min):
            continue
        if len(df_umls_norm.at[y,'STRstem']) > int(num_max):
            continue
        if not df_umls_norm.at[y,'STRstem'][0] == df_termos_stem.at[x,'Termos'][0]:
            continue
        num = levenshtein(df_termos_stem.at[x,'Termos'],df_umls_norm.at[y,'STRstem'])
        if len(df_umls_norm.at[y,'STRstem']) > 0:
            perc = 100-num/len(df_umls_norm.at[y,'STRstem'])*100
        else:
            perc = 0
        if perc == 100:
            achou_Term = True
            candidatos.at[n_candidato,'CUI'] = df_umls_norm.at[y, 'CUI']
            candidatos.at[n_candidato,'Termos'] = df_termos.at[x,'Termos']
            candidatos.at[n_candidato,'STR_UMLS'] = df_umls_norm.at[y, 'STR']
            candidatos.at[n_candidato,'termo_match'] = df_termos_stem.at[x,'Termos']
            candidatos.at[n_candidato, 'metodo'] = 3
            candidatos.at[n_candidato, 'porc'] = perc
            n_candidato += 1
            break
        if perc >= 90:
            candidatos.at[n_candidato,'CUI'] = df_umls_norm.at[y, 'CUI']
            candidatos.at[n_candidato,'Termos'] = df_termos.at[x,'Termos']
            candidatos.at[n_candidato,'STR_UMLS'] = df_umls_norm.at[y, 'STR']
            candidatos.at[n_candidato,'termo_match'] = df_termos_stem.at[x,'Termos']
            candidatos.at[n_candidato, 'metodo'] = 3
            candidatos.at[n_candidato, 'porc'] = perc
            n_candidato += 1
    candidatos = candidatos.sort_values('metodo')
    print('aqui ', df_termos_stem.at[x,'Termos'] , ' --- ', candidatos)
    if len(candidatos['CUI']) == 0:
        df_termos_match.at[zz,'Termos'] = df_termos.at[x,'Termos']
        df_termos_match.at[zz,'Regra'] = 0
        df_termos_match.at[zz,'num_lev'] = 0
        df_termos_match.at[zz, 'CUI'] = ''
        df_termos_match.at[zz, 'STR_UMLS'] = ''
        df_termos_match.at[zz, 'termo_match'] = ''
        zz += 1
    else:
        for y in range(0, len(candidatos['CUI'])):
            df_termos_match.at[zz,'Termos'] = df_termos.at[x,'Termos']
            df_termos_match.at[zz,'Regra'] = candidatos.at[y,'metodo']
            df_termos_match.at[zz,'num_lev'] = candidatos.at[y,'porc']
            df_termos_match.at[zz, 'CUI'] = candidatos.at[y,'CUI']
            df_termos_match.at[zz, 'STR_UMLS'] = candidatos.at[y,'STR_UMLS']
            df_termos_match.at[zz, 'termo_match'] = candidatos.at[y,'termo_match']
            zz += 1
    fimR = time.time()
    print('tempo, ', (fimR-iniR))
    perc_max = 0

servic particul
18.75  e  11.25
aqui  servic particul  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  10.557265996932983
pupil fotorreagent
22.5  e  13.5
aqui  pupil fotorreagent  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  12.557745456695557
non
3.75  e  2.25
aqui  non  ---          CUI metodo porc Termos STR_UMLS termo_match
0  C1010822      3  100   none     noni         non
tempo,  5.218588590621948
drus perimacul
17.5  e  10.5
aqui  drus perimacul  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  10.269019365310669
atrof epiteli pigment retinian
37.5  e  22.5
aqui  atrof epiteli pigment retinian  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  12.432944059371948
non
3.75  e  2.25
aqui  non  ---          CUI metodo porc Termos STR_UMLS termo_match
0  C1010822      3  100   none

aqui  limitaca articul  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  10.125761985778809
medic imunossupressor
26.25  e  15.75
aqui  medic imunossupressor  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  11.419002771377563
cervical anterior-esquerd , posterior
46.25  e  27.75
aqui  cervical anterior-esquerd , posterior  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  14.497195482254028
flux medi
11.25  e  6.75
aqui  flux medi  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  8.65909218788147
vertig noturn
16.25  e  9.75
aqui  vertig noturn  ---  Empty DataFrame
Columns: [CUI, metodo, porc, Termos, STR_UMLS, termo_match]
Index: []
tempo,  9.407527685165405
discret sopr sistol foc mitral
37.5  e  22.5
aqui  discret sopr sistol foc mitral  ---  Empty DataFrame
Columns: [CUI, metodo, porc

In [138]:
writer = pd.ExcelWriter('CliniMap_M3.xlsx')
df_termos_match.to_excel(writer, 'Match')
writer.save()

In [139]:
read = pd.ExcelFile('CliniMap_M3.xlsx')
clinMap_3 = read.parse('Match')

In [140]:
df_termos_match = pd.DataFrame(data=None, columns=['Termos', 'CUI', 'STR_UMLS', 'termo_match', 'Regra', 'num_lev'])

In [ ]:
for x in range(0,len(df_termos_norm)):
    if df_termos_norm.at[x,'Termos'] == None:
        continue
    num_max = len(df_termos_norm.at[x,'Termos'])+len(df_termos_norm.at[x,'Termos'])*0.25
    num_min = len(df_termos_norm.at[x,'Termos'])-len(df_termos_norm.at[x,'Termos'])*0.25
    print(df_termos_norm.at[x,'Termos'])
    print(num_max, ' e ', num_min)
    candidatos = pd.DataFrame(data=None, columns=['CUI', 'metodo', 'porc', 'Termos', 'STR_UMLS', 'termo_match'])
    n_candidato = 0
    words = df_termos_norm.at[x,'Termos'].split()
    words_lemma = df_termos_lemma.at[x,'Termos'].split()
    words_steam = df_termos_stem.at[x, 'Termos'].split()
    
    iniR = time.time()
    for y in range(0,len(df_umls_norm['STR'])):
        if df_umls_norm.at[y,'STR'] == None:
            continue
        if len(df_umls_norm.at[y,'STR']) < int(num_min):
            continue
        if len(df_umls_norm.at[y,'STR']) > int(num_max):
            continue
        if not df_umls_norm.at[y,'STR'][0] == df_termos_norm.at[x,'Termos'][0]:
            continue
        num = levenshtein(df_termos_norm.at[x,'Termos'],df_umls_norm.at[y,'STR'])
        if len(df_umls_norm.at[y,'STR']) > 0:
            perc = 100-num/len(df_umls_norm.at[y,'STR'])*100
        else:
            perc = 0
        if perc == 100:
            achou_Term = True
            candidatos.at[n_candidato,'CUI'] = df_umls_norm.at[y, 'CUI']
            candidatos.at[n_candidato,'Termos'] = df_termos.at[x,'Termos']
            candidatos.at[n_candidato,'STR_UMLS'] = df_umls_norm.at[y, 'STR']
            candidatos.at[n_candidato,'termo_match'] = df_termos_norm.at[x,'Termos']
            candidatos.at[n_candidato, 'metodo'] = 1
            candidatos.at[n_candidato, 'porc'] = perc
            n_candidato += 1
            break
        if perc >= 90:
            candidatos.at[n_candidato,'CUI'] = df_umls_norm.at[y, 'CUI']
            candidatos.at[n_candidato,'Termos'] = df_termos.at[x,'Termos']
            candidatos.at[n_candidato,'STR_UMLS'] = df_umls_norm.at[y, 'STR']
            candidatos.at[n_candidato,'termo_match'] = df_termos_norm.at[x,'Termos']
            candidatos.at[n_candidato, 'metodo'] = 1
            candidatos.at[n_candidato, 'porc'] = perc
            n_candidato += 1
    candidatos = candidatos.sort_values('metodo')
    print('aqui ', df_termos_norm.at[x,'Termos'] , ' --- ', candidatos)
    if len(candidatos['CUI']) == 0:
        df_termos_match.at[zz,'Termos'] = df_termos.at[x,'Termos']
        df_termos_match.at[zz,'Regra'] = 0
        df_termos_match.at[zz,'num_lev'] = 0
        df_termos_match.at[zz, 'CUI'] = ''
        df_termos_match.at[zz, 'STR_UMLS'] = ''
        df_termos_match.at[zz, 'termo_match'] = ''
        zz += 1
    else:
        for y in range(0, len(candidatos['CUI'])):
            df_termos_match.at[zz,'Termos'] = df_termos.at[x,'Termos']
            df_termos_match.at[zz,'Regra'] = candidatos.at[y,'metodo']
            df_termos_match.at[zz,'num_lev'] = candidatos.at[y,'porc']
            df_termos_match.at[zz, 'CUI'] = candidatos.at[y,'CUI']
            df_termos_match.at[zz, 'STR_UMLS'] = candidatos.at[y,'STR_UMLS']
            df_termos_match.at[zz, 'termo_match'] = candidatos.at[y,'termo_match']
            zz += 1
    fimR = time.time()
    print('tempo, ', (fimR-iniR))
    perc_max = 0
cursor.close()
cnx.close()